<div align="center">
    <img src="https://www.sharif.ir/documents/20124/0/logo-fa-IR.png/4d9b72bc-494b-ed5a-d3bb-e7dfd319aec8?t=1609608338755" alt="Logo" width="200">
    <p><b>HW1 @ Deep Learning Course, Dr. Soleymani</b></p>
    <p><b>ِDesinged by Payam Taebi</b></p>
</div>

---




*Full Name:* Mobina Poulaei

*Student Number:* 403206962



# Overview(100 Points)

This notebook is primarily for exploration rather than heavy implementation. We aim to examine the performance of various optimization methods on different functions. The submission for this exercise is in-person, so please experiment with the parameters in this section to better understand the challenges of each part.

We will define several 1D and 2D functions (e.g., Rastrigin, Schwefel, Rosenbrock, Ackley, Beale, Eggholder, Ill-Conditioned Convex functions) using PyTorch. These functions serve as challenging test cases for our optimizers.

Implement different optimization algorithms:
We cover first-order methods (SGD, SGD with Momentum, Nesterov Accelerated Gradient, Adagrad, RMSprop, Adam, Nadam) and second-order methods (Newton's Method, L-BFGS). Each method's update rule is implemented and explained.

These are some of the components we used.


# First Section

In the first section, we have a set of functions for demonstration. Do not modify these; they are implemented to display the functions in this section.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import plotly.graph_objects as go

def plot_1d_function_torch(func, domain, global_x, title="1D Function Plot", initial_x=None, path=None):
    """
    Plot a 1D function (defined in PyTorch) and mark the global minimum.
    Optionally, mark an initial point and the optimization path with directional arrows.

    Parameters:
      - func: a function that accepts a torch tensor and returns a torch tensor.
      - domain: tuple (xmin, xmax)
      - global_x: x-coordinate of the global minimum.
      - title: title for the plot.
      - initial_x: (optional) x-coordinate of the initial point.
      - path: (optional) a list or numpy array of x-values representing the optimization path.
    """
    x_np = np.linspace(domain[0], domain[1], 1000)
    x_torch = torch.tensor(x_np, dtype=torch.float32)
    with torch.no_grad():
        y_torch = func(x_torch)
    y_np = y_torch.numpy()

    plt.figure(figsize=(8, 4))
    plt.plot(x_np, y_np, lw=2, label=title)

    # Mark initial point if provided.
    if initial_x is not None:
        init_val = func(torch.tensor(initial_x, dtype=torch.float32)).item()
        plt.scatter(initial_x, init_val, color='blue', s=80, zorder=5, label='Initial Point')

    # Plot the optimization path if provided.
    if path is not None:
        path = np.array(path)
        # Draw a line connecting the points.
        y_path = [func(torch.tensor(p, dtype=torch.float32)).item() for p in path]
        plt.plot(path, y_path, color='orange', linewidth=2, marker='o', markersize=4, label='Optimization Path')
        # Draw arrows for each segment.
        for i in range(len(path)-1):
            start = path[i]
            end = path[i+1]
            y_start = func(torch.tensor(start, dtype=torch.float32)).item()
            y_end = func(torch.tensor(end, dtype=torch.float32)).item()
            plt.annotate("",
                         xy=(end, y_end),
                         xytext=(start, y_start),
                         arrowprops=dict(arrowstyle="->", color='orange', lw=2))

    # Mark the global minimum.
    global_val = func(torch.tensor(global_x, dtype=torch.float32)).item()
    plt.scatter(global_x, global_val, color='red', s=80, zorder=5, label='Global Minimum')

    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_2d_contour_torch(func, x_domain, y_domain, global_point, title="2D Function Contour",
                            levels=50, cmap='viridis', initial_point=None, path=None):
    """
    Plot a 2D function (defined in PyTorch) using a contour plot, marking the global minimum.
    Optionally, mark an initial point and draw the optimization path with arrows indicating direction.

    Parameters:
      - func: a function that accepts a torch tensor of shape (2, H, W) and returns a tensor of shape (H, W).
      - x_domain: tuple (xmin, xmax)
      - y_domain: tuple (ymin, ymax)
      - global_point: tuple (x*, y*) of the global minimum.
      - title: title for the plot.
      - levels: number of contour levels.
      - cmap: colormap.
      - initial_point: (optional) tuple (x0, y0) for the initial point.
      - path: (optional) numpy array of shape (n, 2) representing the optimization path.
    """
    x_np = np.linspace(x_domain[0], x_domain[1], 400)
    y_np = np.linspace(y_domain[0], y_domain[1], 400)
    X_np, Y_np = np.meshgrid(x_np, y_np)

    # Create input grid tensor with shape (2, H, W)
    grid = torch.tensor(np.stack([X_np, Y_np], axis=0), dtype=torch.float32)
    with torch.no_grad():
        Z_torch = func(grid)
    Z_np = Z_torch.numpy()
    vmin, vmax = Z_np.min(), Z_np.max()

    plt.figure(figsize=(10, 8))
    cp = plt.contourf(X_np, Y_np, Z_np, levels=levels, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar(cp)
    plt.contour(X_np, Y_np, Z_np, levels=15, colors='black', alpha=0.5)

    # Mark initial point if provided.
    if initial_point is not None:
        plt.scatter(initial_point[0], initial_point[1], color='blue', s=100, marker='o', label='Initial Point')

    # Plot the optimization path if provided.
    if path is not None:
        path = np.array(path)
        plt.plot(path[:, 0], path[:, 1], marker='o', color='orange', markersize=4, linewidth=2, label='Optimization Path')
        for i in range(len(path) - 1):
            start = path[i]
            end = path[i+1]
            plt.annotate("",
                         xy=(end[0], end[1]),
                         xytext=(start[0], start[1]),
                         arrowprops=dict(arrowstyle="->", color='orange', lw=2))

    # Mark the global minimum.
    plt.scatter(global_point[0], global_point[1], color='red', s=100, marker='*', label='Global Minimum')

    plt.title(title + " (Contour Plot)")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()

def plot_interactive_3d_torch(func, x_domain, y_domain, global_point, title="Interactive 3D Plot",
                              colorscale='viridis', initial_point=None, path=None):
    """
    Create an interactive 3D surface plot using Plotly for a 2D function defined in PyTorch.
    Assumes the input is a tensor with two channels: first channel for x and second for y.
    Optionally, marks an initial point and draws cones (arrows) to indicate direction along the optimization path.

    Parameters:
      - func: a function that accepts a torch tensor of shape (2, H, W) and returns a tensor of shape (H, W).
      - x_domain: tuple (xmin, xmax)
      - y_domain: tuple (ymin, ymax)
      - global_point: tuple (x*, y*) for the global minimum.
      - title: title for the plot.
      - colorscale: Plotly colorscale.
      - initial_point: (optional) tuple (x0, y0) for the initial point.
      - path: (optional) numpy array of shape (n, 2) representing the optimization path.
    """
    # Create grid for the surface.
    x_np = np.linspace(x_domain[0], x_domain[1], 200)
    y_np = np.linspace(y_domain[0], y_domain[1], 200)
    X_np, Y_np = np.meshgrid(x_np, y_np)

    # Create grid tensor with shape (2, H, W)
    grid = torch.tensor(np.stack([X_np, Y_np], axis=0), dtype=torch.float32)
    with torch.no_grad():
        Z_torch = func(grid)
    Z_np = Z_torch.numpy()

    surface = go.Surface(x=X_np, y=Y_np, z=Z_np, colorscale=colorscale, opacity=0.9, name=title)
    data = [surface]

    # Optionally add initial point.
    if initial_point is not None:
        init_val = func(torch.tensor([initial_point[0], initial_point[1]], dtype=torch.float32)).item()
        scatter_initial = go.Scatter3d(
            x=[initial_point[0]],
            y=[initial_point[1]],
            z=[init_val],
            mode='markers',
            marker=dict(size=6, color='blue'),
            name='Initial Point'
        )
        data.append(scatter_initial)

    # Optionally add optimization path.
    if path is not None:
        path = np.array(path)
        z_path = []
        for pt in path:
            z_val = func(torch.tensor([pt[0], pt[1]], dtype=torch.float32)).item()
            z_path.append(z_val)
        scatter_path = go.Scatter3d(
            x=path[:, 0],
            y=path[:, 1],
            z=z_path,
            mode='lines+markers',
            line=dict(color='orange', width=4),
            marker=dict(size=4, color='orange'),
            name='Optimization Path'
        )
        data.append(scatter_path)

        # Add cone traces for directional arrows.
        cone_x = []
        cone_y = []
        cone_z = []
        cone_u = []
        cone_v = []
        cone_w = []

        for i in range(len(path) - 1):
            start = path[i]
            end = path[i+1]
            cone_x.append(start[0])
            cone_y.append(start[1])
            # Evaluate z at start.
            z_start = func(torch.tensor([start[0], start[1]], dtype=torch.float32)).item()
            cone_z.append(z_start)
            cone_u.append(end[0] - start[0])
            cone_v.append(end[1] - start[1])
            z_end = func(torch.tensor([end[0], end[1]], dtype=torch.float32)).item()
            cone_w.append(z_end - z_start)

        if len(cone_x) > 0:
            cone_trace = go.Cone(
                x=cone_x,
                y=cone_y,
                z=cone_z,
                u=cone_u,
                v=cone_v,
                w=cone_w,
                sizemode="absolute",
                sizeref=0.5,
                anchor="tail",
                showscale=False,
                colorscale=[[0, 'orange'], [1, 'orange']],
                name="Step Arrows"
            )
            data.append(cone_trace)

    # Add global minimum.
    global_tensor = torch.tensor([global_point[0], global_point[1]], dtype=torch.float32)
    with torch.no_grad():
        global_z = func(global_tensor).item()
    scatter_global = go.Scatter3d(
        x=[global_point[0]],
        y=[global_point[1]],
        z=[global_z],
        mode='markers',
        marker=dict(size=8, color='red', symbol='diamond'),
        name='Global Minimum'
    )
    data.append(scatter_global)

    fig = go.Figure(data=data)
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title='f(x,y)'
        ),
        autosize=True
    )
    fig.show()


def plot_loss_vs_epoch(loss_func, path):

    """
    Given a loss function and an optimization path, compute the loss at each epoch and plot it.

    Parameters:
      - loss_func: A function that accepts a torch tensor representing parameters and returns a scalar loss.
      - path: A list or NumPy array of parameter values (e.g. from the optimizer).
              Each element should be convertible to a torch tensor.
    """
    losses = []
    # Loop over each parameter value in the path.
    for p in path:
        # Convert parameter p to a torch tensor (assume float32).
        # p can be a 1D array or a multi-dimensional array.
        p_tensor = torch.tensor(p, dtype=torch.float32)
        with torch.no_grad():
            loss_val = loss_func(p_tensor).item()
        losses.append(loss_val)

    epochs = np.arange(len(losses))

    plt.figure(figsize=(8, 4))
    plt.plot(epochs, losses, marker='o', color='blue', linewidth=2)
    plt.title("Loss vs. Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.show()

# Next Section (18 Points)

In the following section, we examine all the functions, each of which presents its own unique challenge. These functions are both one-dimensional and two-dimensional.


# 1D Rastrigin Function

**Mathematical Definition:**

$$
f(x) = x^2 - 10 \cos\left(2\pi x\right) + 10
$$

**Domain:**  
\\( x \in [-5,\, 5] \\)

**Global Minimum:**  
\\( x = 0 \\) with \\( f(0)=0 \\)

**Explanation:**  
This function is **non-convex** and exhibits many local minima due to the cosine modulation. Its oscillatory behavior over \\( [-5,5] \\) makes it a popular test for global optimization methods.


In [ ]:
# 1D Rastrigin function in PyTorch.
def rastrigin_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D Rastrigin function using PyTorch operations
    return x**2 - 10 * torch.cos(2 * np.pi * x) + 10

# Plot the 1D Rastrigin function.
plot_1d_function_torch(rastrigin_1d_torch, [-5, 5], global_x=0, title="1D Rastrigin Function")

# 1D Schwefel Function

**Mathematical Definition:**

$$
f(x) = 418.9829 - x \sin\Bigl(\sqrt{|x|}\Bigr)
$$

**Domain:**  
\\( x \in [0,\, 500] \\)

**Global Minimum:**  
Approximately at \\( x \approx 420.9687 \\)

**Explanation:**  
This function is highly **non-convex** and rugged with many deceptive local minima. Its large domain accentuates the difficulty in escaping local traps, making it a challenging benchmark for optimization.


In [ ]:
# 1D Schwefel function in PyTorch.
def schwefel_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D Schwefel function using PyTorch operations
    return 418.9829 - x * torch.sin(torch.sqrt(torch.abs(x)))

# Plot the 1D Schwefel function.
plot_1d_function_torch(schwefel_1d_torch, [0, 500], global_x=420.9687, title="1D Schwefel Function")

# 1D Ill-Conditioned Convex Function

**Mathematical Definition:**

$$
f(x) = \ln\Bigl(1 + e^{10x}\Bigr) + x^2
$$

**Domain:**  
\\( x \in [-5,\, 5] \\)

**Global Minimum:**  
Approximately at \\( x \approx -0.3 \\)

**Explanation:**  
This function is **convex** (and therefore has a unique global minimum), but it is _ill-conditioned_ because the steep exponential part for \\( x > 0 \\) creates a rapidly changing gradient. It tests how well an optimizer can adjust its step size.


In [ ]:
# 1D Ill-Conditioned Convex function in PyTorch.
def ill_conditioned_convex_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D ill-conditioned convex function using PyTorch operations
    return torch.log(1 + torch.exp(10*x)) + x**2

# Plot the 1D Ill-Conditioned Convex function.
plot_1d_function_torch(ill_conditioned_convex_1d_torch, [-5, 5], global_x=-0.3, title="1D Ill-Conditioned Convex Function")

# Rosenbrock Function (2D)

**Mathematical Definition:**

$$
f(x, y) = (1 - x)^2 + 100 \,(y - x^2)^2
$$

**Domain:**  
\\( x \in [-2,\, 2] \\) and \\( y \in [-1,\, 3] \\)

**Global Minimum:**  
At \\( (x,y) = (1,1) \\) with \\( f(1,1)=0 \\)

**Explanation:**  
The Rosenbrock function is **non-convex** despite its smoothness. Its narrow, curved valley makes it challenging for gradient-based optimizers to converge to the global minimum. This function is a classical benchmark for testing optimization algorithms.

In the next cells, we plot it both as a static contour and as an interactive 3D plot. We'll also mark an initial point (for example, \\( (-1,1.5) \\)) and the optimization path.


In [ ]:
# Define the Rosenbrock function for 2D inputs in PyTorch.
# The input is a tensor of shape (2, H, W) where the first channel is x and the second is y.
def rosenbrock_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Rosenbrock function using PyTorch operations
    return (1 - X[0])**2 + 100 * (X[1] - X[0]**2)**2

# Plot using static contour.
plot_2d_contour_torch(rosenbrock_2d_torch, [-2, 2], [-1, 3], global_point=(1, 1),
                      title="Rosenbrock Function (2D)")

# Plot using interactive 3D.
plot_interactive_3d_torch(rosenbrock_2d_torch, [-2, 2], [-1, 3], global_point=(1, 1),
                          title="Interactive 3D Rosenbrock Function")

# 2D Rastrigin Function

**Mathematical Definition:**

$$
f(x, y) = x^2 + y^2 - 10 \Bigl[\cos\left(2\pi x\right) + \cos\left(2\pi y\right)\Bigr] + 20
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
The 2D Rastrigin function is highly **non-convex** and multimodal, featuring many local minima. Its oscillatory behavior in both dimensions makes it a demanding test for optimization techniques.

Below, we display both the static contour and interactive 3D plots with an example initial point (e.g., \\( (3,3) \\)) and a dummy optimization path.


In [ ]:
# Define the 2D Rastrigin function in PyTorch.
def rastrigin_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Rastrigin function using PyTorch operations
    return X[0]**2 + X[1]**2 - 10 * (torch.cos(2 * np.pi * X[0]) + torch.cos(2 * np.pi * X[1])) + 20

# Plot as a static contour.
plot_2d_contour_torch(rastrigin_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="2D Rastrigin Function", cmap='plasma')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(rastrigin_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Rastrigin Function", colorscale='plasma')

# Ackley Function (2D)

**Mathematical Definition:**

$$
f(x, y) = -20 \exp\!\left(-0.2 \sqrt{\frac{x^2+y^2}{2}}\right) - \exp\!\left(\frac{\cos\left(2\pi x\right)+\cos\left(2\pi y\right)}{2}\right) + 20 + e
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
The Ackley function features a nearly flat outer region with a deep, narrow central pit, creating a challenging landscape for optimization algorithms. This function is popular for testing global optimization techniques.

We now show both the contour and interactive 3D plots with an initial point (e.g., \\( (3,3) \\)) and an optimization path.


In [ ]:
# Define the 2D Ackley function in PyTorch.
def ackley_2d_torch(X: torch.Tensor, a=20, b=0.2, c=2*np.pi) -> torch.Tensor:
    # TODO: Implement the 2D Ackley function using PyTorch operations
    return -a * torch.exp(-b * torch.sqrt(0.5 * (X[0]**2 + X[1]**2))) - torch.exp(0.5 * (torch.cos(c * X[0]) + torch.cos(c * X[1]))) + a + torch.exp(torch.tensor(1))

# Plot as a static contour.
plot_2d_contour_torch(ackley_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="Ackley Function (2D)", cmap='inferno')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(ackley_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Ackley Function", colorscale='inferno')

# Beale Function (2D)

**Mathematical Definition:**

$$
f(x, y) = \left(1.5 - x + xy\right)^2 + \left(2.25 - x + xy^2\right)^2 + \left(2.625 - x + xy^3\right)^2
$$

**Domain:**  
\\( x, y \in [-4.5,\, 4.5] \\)

**Global Minimum:**  
At \\( (3, 0.5) \\) with \\( f(3,0.5)=0 \\)

**Explanation:**  
The Beale function is **non-convex** with several local minima, making it a classical benchmark for global optimization. Its complex structure challenges optimizers in 2D space.

Below, we provide both a static contour and an interactive 3D plot with an initial point (e.g., \\( (0,0) \\)) and an optimization path.


In [ ]:
# Define the 2D Beale function in PyTorch.
def beale_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Beale function using PyTorch operations
    return (1.5 - X[0] + X[0] * X[1])**2 + (2.25 - X[0] + X[0] * X[1]**2)**2 + (2.625 - X[0] + X[0] * X[1]**3)**2

# Plot as a static contour.
plot_2d_contour_torch(beale_2d_torch, [-4.5, 4.5], [-4.5, 4.5], global_point=(3, 0.5),
                      title="Beale Function (2D)", cmap='Spectral')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(beale_2d_torch, [-4.5, 4.5], [-4.5, 4.5], global_point=(3, 0.5),
                          title="Interactive 3D Beale Function", colorscale='Spectral')

# Eggholder Function (2D)

**Mathematical Definition:**

$$
f(x, y) = - (y+47) \sin\!\left(\sqrt{\left|\frac{x}{2} + (y+47)\right|}\right) - x \sin\!\left(\sqrt{\left|x - (y+47)\right|}\right)
$$

**Domain:**  
\\( x, y \in [-512,\, 512] \\)

**Global Minimum:**  
Approximately at \\( (512,\, 404.2319) \\)

**Explanation:**  
The Eggholder function is extremely **non-convex** with a rugged, highly oscillatory landscape that contains many local minima. It is one of the most challenging test functions in optimization.


In [ ]:
# Define the 2D Eggholder function in PyTorch.
def eggholder_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Eggholder function using PyTorch operations
    return - (X[1] + 47) * torch.sin(torch.sqrt(torch.abs((X[0] / 2) + (X[1] + 47)))) - X[0] * torch.sin(torch.sqrt(torch.abs(X[0] - (X[1] + 47))))

# Plot as a static contour.
plot_2d_contour_torch(eggholder_2d_torch, [-512, 512], [-512, 512], global_point=(512, 404.2319),
                      title="Eggholder Function (2D)", cmap='inferno')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(eggholder_2d_torch, [-512, 512], [-512, 512], global_point=(512, 404.2319),
                          title="Interactive 3D Eggholder Function", colorscale='inferno')

# Ill-Conditioned Convex Function (2D)

**Mathematical Definition:**

$$
f(x, y) = 100 \Bigl(x\cos\theta - y\sin\theta\Bigr)^2 + \Bigl(x\sin\theta + y\cos\theta\Bigr)^2 \quad \text{with } \theta = \frac{\pi}{6}
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
This function is **convex** (thus it has a unique global minimum) but is _ill-conditioned_ due to the rotation and different scaling along the rotated axes. Its behavior tests an optimizer’s ability to deal with steep versus flat directions.


In [ ]:
# Define the 2D Ill-Conditioned Convex function in PyTorch.
def ill_conditioned_convex_2d_torch(X: torch.Tensor, theta=torch.tensor(np.pi/6)) -> torch.Tensor:
    # TODO: Implement the 2D ill-conditioned convex function using PyTorch operations
    return 100*(X[0]*torch.cos(theta) - X[1]*torch.sin(theta))**2 + (X[0]*torch.sin(theta) + X[1]*torch.cos(theta))**2

# Plot as a static contour.
plot_2d_contour_torch(ill_conditioned_convex_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="Ill-Conditioned Convex Function (2D)", cmap='viridis')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(ill_conditioned_convex_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Ill-Conditioned Convex Function (2D)", colorscale='viridis')

# Question (12 Points)

For each of the functions presented above, what unique challenges do you think they offer? Consider aspects such as steep gradients, saddle points, local minima, flat regions, and other characteristics that could affect the optimization process.

Please provide a brief explanation for each function, describing the potential difficulties in learning or optimizing it.




#Answers
---
###1D Rastrigin


1.   **Many Local Minima**: The function is highly multimodal because of its cosine term. This creates a series of regularly distributed local minima that can easily trap optimization algorithms.

2.   **Initialization Sensitivity**: The starting point can have a large effect on the outcome. A poor initialization might land an algorithm in a region dense with local optima, making it challenging to escape.

3. **Non-convexity Issues**: The function’s non-convex nature means that standard gradient descent methods can struggle, as they may follow the steepest descent into a local minimum and fail to move out.

4. **Oscillatory Gradients**: The periodic cosine component creates oscillatory gradient behavior, which can make step size selection crucial. Too large a step might skip over minima, while too small a step may result in slow convergence.

---
###1D Schwefel
1. **Many Local Optima:** The Schwefel function is notorious for its large number of local minima. Even in one dimension, the oscillatory nature of the function creates many deceptive optima, making it challenging for algorithms to identify the true global minimum.

2. **Non-Convexity**: The inherent non-convexity of the Schwefel function means that small local changes in the search space do not necessarily lead toward the global optimum. Standard gradient methods may therefore be misled by local landscape features.

3. **Steep Gradients**: The sine-based component introduces areas with steep gradients as well as regions where the gradient changes rapidly. This variability makes it difficult for gradient-based algorithms to choose an appropriate step size consistently.

4. **Boundary Effects**: A unique challenge for the Schwefel function is that its global optimum is often located near the boundaries of the search space. This means that algorithms must explore the edges effectively or they risk missing the optimum.



---
###1D Ill-Conditioned Convex
1. **Steep and Nearly Flat Regions**: Although the function is convex and has a unique minimum, its ill-conditioning implies that there can be regions with extremely steep gradients alongside areas that are nearly flat. This disparity can cause difficulties in selecting an appropriate step size for optimization algorithms.

2. **Overshooting in Steep Regions**: Conversely, in regions with very steep gradients, an algorithm might overshoot the optimum if the step size is not carefully controlled, which can destabilize the convergence process.


---
###2D Rosenbrock
1. **Curved Valley**: The global minimum lies inside a long, narrow, curved valley (often referred to as the "banana" shape). This makes it challenging for optimizers to converge quickly because they must navigate along the curved trajectory rather than heading directly toward the minimum.

2. **Ill-Conditioning**: The Hessian matrix of the function is badly conditioned near the optimum, leading to slow convergence in optimization methods like Newton’s method.

3. **Gradient Behavior**: The gradient is small in the flat regions of the valley and changes rapidly near the curved walls, which challenges the optimizer to balance exploration.

---
### 2D Rastrigin

1. **High Multimodality**: The 2D Rastrigin function is characterized by a vast number of local minima due to its cosine components in both dimensions. This rugged landscape increases the risk of getting trapped in suboptimal solutions.

2. **Oscillatory Behavior**: The superimposed cosine terms introduce periodic oscillations in the function's landscape. This creates rapidly changing gradients that can mislead gradient-based methods and complicate step size determination.

3. **Dimensional Complexity**: In two dimensions, the coupling between the oscillatory patterns of each axis adds to the complexity. The interactions can create intricate basins of attraction.

---
### 2D Ackley
1. **Steep Central Basin**: Near the global optimum, the landscape becomes steeper and more curved. Optimizers must balance careful navigation in these steep regions without overshooting the optimum.

2. **Multimodal Landscape**: The presence of many local minima in addition to the global minimum creates a highly rugged and complex landscape. This increases the risk of optimization algorithms getting trapped in suboptimal solutions, particularly for local search methods or gradient-based algorithms.

3. **Flat Outer Regions**: The Ackley function has flat regions (especially away from the global optimum) and steep areas near the central optimum. The optimizer needs to be able to escape the flat regions without overshooting the steep, more rapidly changing parts.


---
### 2D Beale
1. **Flat Regions and Plateaus**: Portions of the function can be relatively flat, causing low gradient magnitudes. These flat regions may slow convergence as the optimizer struggles to identify a clear direction for descent.

2. **Sensitivity to Initialization**: The success of finding the global optimum is heavily reliant on the starting point. Poor initialization might lead the algorithm into areas where the steep gradients or flat regions hinder effective progress.

---
### 2D Eggholder
1. **Extreme Multimodality**: The Eggholder function is infamous for its large number of local minima. This highly rugged landscape can easily mislead optimization algorithms.

2. **Highly Irregular Surface**: The function’s landscape is characterized by deep valleys and high ridges with rapidly changing gradients. Such behavior poses significant difficulties for gradient-based methods, which may struggle with step-size selection.

3. **Interdependent Variables**: The variables in the Eggholder function are coupled in a complex way, meaning that the behavior in one dimension directly affects the other. This interdependence increases the difficulty of isolating and correcting errors during optimization.

---
### 2D Ill-Conditioned Convex
1. **Disparity in Curvature**: The function has a significant difference in curvature along different directions (one direction is much steeper than the other). This disparity creates a narrow valley where the global minimum lies, making it difficult for algorithms to move efficiently.

2. **Slow Convergence Along Flat Directions**: In the nearly flat directions, the gradients are very small, which can result in slow progress and require many iterations for convergence.

3. **Overshooting in Steep Directions**: The steep direction can lead to overly aggressive updates. Without proper step size control, the optimizer may overshoot the minimum, causing oscillations or divergence.

4. **Sensitivity to Scaling and Hyperparameters**: The ill-conditioning means that small changes in the learning rate or other hyperparameters can significantly impact convergence behavior. Proper scaling, adaptive step sizes, or preconditioning techniques are often necessary to achieve efficient optimization.

# Implementing an Optimization Method (5 Points)

Now that we've discussed the general structure of our optimization function, let's explore how to implement a specific optimization method. In this section, we'll walk through the steps required to define an update rule for an optimizer. This involves:

1. **Computing the Loss and its Derivatives:**  
   We evaluate the loss function for the current parameters, compute the gradient, and optionally calculate the Hessian if using a second-order method.

2. **Applying the Update Rule:**  
   Based on the computed derivatives, we update the parameters using the specific logic of our chosen optimization method (e.g., adjusting the parameters using SGD, Adam, or Newton's method).

3. **Maintaining State:**  
   For methods that require momentum or other historical data, we update and maintain a state dictionary that carries this information from one iteration to the next.

By following these steps, we can iteratively update the parameters to minimize the loss function and analyze the optimization trajectory. Let's move on to see this process in action.

# General Optimization Function Explanation

The `optimize` function provides a unified framework for iterative optimization methods. Whether using first-order (gradient-based) or second-order (Hessian-based) methods, the overall process remains similar: we start with an initial guess for the parameters and then iteratively update these parameters in order to minimize a given loss function.

## How It Works

1. **Initialization:**
   - **Parameters:**  
     The function receives an initial parameter tensor (`initial_params`). This tensor is cloned and set to require gradients so that we can compute derivatives with respect to it.
   - **State Storage:**  
     An empty dictionary `state` is initialized to store any additional information required by the update method (e.g., momentum for Adam, or curvature approximations for second-order methods).
   - **Path Recording:**  
     The initial parameters are stored in a list called `path`. This list will hold the parameter values after each iteration, allowing us to visualize the optimization trajectory later.

2. **Iterative Update Process:**
   - For each epoch (i.e., optimization step), the following steps occur:
     1. **Loss Evaluation:**  
        The loss function `loss_func` is evaluated at the current parameter values to determine how well the current parameters perform.
     2. **Gradient Computation:**  
        The gradient (first derivative) of the loss with respect to the parameters is computed using `loss.backward()`.  
        - For **first-order methods**, only the gradient is used.
        - For **second-order methods**, additional curvature information is needed.
     3. **Hessian Computation (Optional):**  
        If the `second_order` flag is set to `True`, the Hessian (matrix of second derivatives) is computed using PyTorch's `torch.autograd.functional.hessian`.  
        This Hessian provides insight into the curvature of the loss surface and is useful for methods like Newton's method.
     4. **Parameter Update:**  
        The update function `update_func` is then called:
        - For **first-order methods** (e.g., SGD, Adam):  
          It is called with the current parameters, their gradient, the state, and hyperparameters.
        - For **second-order methods** (e.g., Newton’s Method, L-BFGS):  
          The computed Hessian is also passed.
        This function returns updated parameter values and an updated state.
     5. **Re-enable Gradient Tracking:**  
        After the update, gradients are cleared and re-enabled for the next iteration.
     6. **Path Recording:**  
        The new parameters are added to the `path` list.

3. **Return Value:**
   - After completing all epochs, the function returns the `path` — a list of parameter values at each iteration. This path can later be used to visualize the optimization trajectory (for example, by plotting the loss versus epochs or overlaying the path on the loss surface).

## Common Underlying Principles

- **Unified Structure:**  
  All optimization methods follow a similar iterative procedure: compute the loss, compute the gradient (and optionally the Hessian), update the parameters, and record the progress.
  
- **First-Order vs. Second-Order Methods:**  
  - *First-Order Methods* use only the gradient information. Their update rule typically has the form:  
  $$
    w_{t+1} = w_t - \eta \nabla f(w_t)
  $$
  - *Second-Order Methods* also incorporate curvature information (via the Hessian) to adjust the step direction and size:  
    $$ w_{t+1} = w_t - \left( H(w_t) + \epsilon I \right)^{-1} \nabla f(w_t) $$
  The `second_order` flag in our function determines whether the Hessian is computed and passed to the update function.

- **State Management:**  
  Many advanced optimizers (like Adam, RMSprop, or L-BFGS) maintain a state that includes historical information (e.g., moving averages of gradients, momentum, or curvature approximations). This state is updated at each iteration and passed to the update function to help guide the parameter updates.

In summary, the `optimize` function abstracts the common iterative process of optimization, allowing us to plug in different update functions (each corresponding to a different optimization method) and compare their performance on the same problem. This modular approach makes it straightforward to experiment with a wide range of optimization algorithms.


In [11]:
import torch

def optimize(loss_func, update_func, initial_params, epochs, hyperparams, second_order=False):
    """
    General optimization function supporting both first-order (gradient-based)
    and second-order (Hessian-based) optimization methods.

    Parameters:
        - loss_func: Function to minimize. Accepts parameters and returns loss.
        - update_func: Function that updates parameters.
                       Signature (first-order): (params, grad, state, hyperparams) -> (new_params, new_state)
                       Signature (second-order): (params, grad, state, hyperparams, hessian) -> (new_params, new_state)
        - initial_params: Torch tensor representing the initial parameter values.
        - epochs: Number of optimization steps.
        - hyperparams: Dictionary of hyperparameters.
        - second_order: Boolean flag indicating if second-order derivatives (Hessian) are needed.

    Returns:
        - path: List of parameter values at each step.
    """
    params = initial_params.clone().detach().requires_grad_(True)
    state = {}  # Store state (e.g., momentum, Hessian approximations, etc.)
    path = [params.clone().detach().numpy()]  # Store initial point

    for epoch in range(epochs):
        loss = loss_func(params)  # Compute loss
        loss.backward()  # Compute gradient

        with torch.no_grad():  # Disable gradient tracking during updates
            if second_order:
                # TODO: Compute the Hessian matrix using torch.autograd.functional.hessian
                hessian = torch.autograd.functional.hessian(loss_func, params)
                # TODO: Update parameters using the update_func with second-order information
                params, state = update_func(params, params.grad, state, hyperparams, hessian)
            else:
                # TODO: Update parameters using the update_func with first-order information
                params, state = update_func(params, params.grad, state, hyperparams)

            params.requires_grad_(True)

        path.append(params.clone().detach().numpy())

    return path

# Optimization Demonstration Functions

Next, we have a set of functions that display the optimization process by running it on all the functions. You may edit them if needed for better visualization, but preferably, do not modify these two sections.


In [12]:
def optimize_and_plot(loss_func, update_func, initial_params, epochs, hyperparams, x_domain, y_domain=None, title="Optimization" ,second_order=False):
    """
    Perform optimization and automatically visualize the results for 1D or 2D functions.

    Parameters:
      - loss_func: Function to minimize. It accepts parameters and returns a scalar loss.
      - update_func: Function that updates parameters. Signature: (params, grad, state, hyperparams) -> (new_params, new_state)
      - initial_params: Torch tensor representing the initial parameter values.
      - epochs: Number of optimization steps.
      - hyperparams: Dictionary of hyperparameters for the update function.
      - x_domain: Tuple (xmin, xmax) defining the domain for plotting.
      - y_domain: Tuple (ymin, ymax) defining the domain for 2D functions (set to None for 1D).
      - title: Title for the plots.

    Returns:
      - path: The list of parameter values at each epoch (for further processing if desired).
    """
    # Run the optimization (assumes a separate "optimize" function exists).
    path = optimize(loss_func, update_func, initial_params, epochs, hyperparams , second_order=second_order)
    path_np = np.array(path)  # Convert list of torch tensors (or arrays) to a NumPy array.
    initial_np = initial_params.detach().numpy()  # Get initial point as a NumPy array.

    # Check dimensionality: if initial_params has shape (1, ...) then it's 1D; if (2, ...) then 2D.
    if initial_params.shape[0] == 1:
        # 1D case: Pass initial_x and path.
        plot_1d_function_torch(
            func=loss_func,
            domain=x_domain,
            global_x=path_np[-1][0],
            title=title,
            initial_x=initial_np[0],
            path=path_np
        )
        plot_loss_vs_epoch(loss_func, path_np)
    elif initial_params.shape[0] == 2:
        # 2D case: Pass initial_point as a tuple and the full path.
        plot_2d_contour_torch(
            func=loss_func,
            x_domain=x_domain,
            y_domain=y_domain,
            global_point=(path_np[-1][0], path_np[-1][1]),
            title=title,
            initial_point=(initial_np[0], initial_np[1]),
            path=path_np
        )
        plot_interactive_3d_torch(
            func=loss_func,
            x_domain=x_domain,
            y_domain=y_domain,
            global_point=(path_np[-1][0], path_np[-1][1]),
            title=title,
            initial_point=(initial_np[0], initial_np[1]),
            path=path_np
        )
        plot_loss_vs_epoch(loss_func, path_np)
    else:
        raise ValueError("Function must be either 1D or 2D.")

    return


In [13]:
def run_all_optimizations(update_func, hyperparams, epochs, second_order=False):
    """
    Run optimization for all benchmark functions (both 1D and 2D) using the provided update function,
    hyperparameters, and epoch count. It calls `optimize_and_plot` to handle visualization.

    Parameters:
      - update_func: The update function to use (e.g., adam_update, sgd_update, etc.).
      - hyperparams: Dictionary of hyperparameters for the update function.
      - epochs: Number of optimization steps to perform.
    """
    # List of benchmark function configurations.
    configs = [
        # 1D Functions
        {
            "title": "1D Rastrigin Function Optimization",
            "loss_func": rastrigin_1d_torch,
            "initial_params": torch.tensor([3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": None
        },
        {
            "title": "1D Schwefel Function Optimization",
            "loss_func": schwefel_1d_torch,
            "initial_params": torch.tensor([100.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (0, 500),
            "y_domain": None
        },
        {
            "title": "1D Ill-Conditioned Convex Function Optimization",
            "loss_func": ill_conditioned_convex_1d_torch,
            "initial_params": torch.tensor([3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": None
        },
        # 2D Functions
        {
            "title": "2D Rosenbrock Function Optimization",
            "loss_func": rosenbrock_2d_torch,
            "initial_params": torch.tensor([-1.0, 1.5], dtype=torch.float32, requires_grad=True),
            "x_domain": (-2, 2),
            "y_domain": (-1, 3)
        },
        {
            "title": "2D Rastrigin Function Optimization",
            "loss_func": rastrigin_2d_torch,
            "initial_params": torch.tensor([3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        },
        {
            "title": "2D Ackley Function Optimization",
            "loss_func": ackley_2d_torch,
            "initial_params": torch.tensor([3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        },
        {
            "title": "2D Beale Function Optimization",
            "loss_func": beale_2d_torch,
            "initial_params": torch.tensor([-3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-4.5, 4.5),
            "y_domain": (-4.5, 4.5)
        },
        {
            "title": "2D Eggholder Function Optimization",
            "loss_func": eggholder_2d_torch,
            "initial_params": torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-512, 512),
            "y_domain": (-512, 512)
        },
        {
            "title": "2D Ill-Conditioned Convex Function Optimization",
            "loss_func": ill_conditioned_convex_2d_torch,
            "initial_params": torch.tensor([4.0, -3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        }
    ]

    # Iterate over each function and call optimize_and_plot
    for config in configs:
        print("Optimizing:", config["title"])

        optimize_and_plot(
            loss_func=config["loss_func"],
            update_func=update_func,
            initial_params=config["initial_params"],
            epochs=epochs,
            hyperparams=hyperparams,
            x_domain=config["x_domain"],
            y_domain=config["y_domain"],
            title=config["title"],
            second_order=second_order
        )

        print("-" * 60)

# Implementation of Optimization Algorithms

Now that the optimization functions have been implemented, in this section we will implement various optimization algorithms. Some parts of these algorithms need to be implemented by you, while other parts have already been provided.

For each algorithm, experiment with different parameters and fine-tune them to achieve the best possible performance. The selection and tuning of these parameters are part of your grade, and the number of times you experiment with various settings is a crucial component of this exercise.

After implementing each algorithm, please submit a report detailing your observations and findings. Make sure to document your parameter tuning process and the performance outcomes for each algorithm.


# Stochastic Gradient Descent (SGD)(5 Points)

**Mathematical Formula:**

$$
w_{t+1} = w_t - \eta \nabla f(w_t)
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate, and
- \\(\nabla f(w_t)\\) is the gradient of the loss function evaluated at \\(w_t\\).

**Explanation:**

Stochastic Gradient Descent (SGD) is a fundamental first-order optimization method that updates the parameters in the opposite direction of the gradient. At each iteration, the update rule moves the parameters by a small step proportional to the gradient and the learning rate. This method is widely used because of its simplicity and efficiency, particularly when dealing with large datasets. However, its performance heavily depends on the proper tuning of the learning rate; if set too high, the algorithm might overshoot minima, and if too low, convergence can be very slow.


In [14]:
def sgd_update(params, grad, state, hyperparams, **kwargs):
    """
    SGD update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store any state (unused in SGD).
      - hyperparams: Dictionary of hyperparameters. Must contain "lr" (learning rate).
      - **kwargs: Additional arguments (ignored for SGD).

    Returns:
      - new_params: Updated parameters.
      - state: Unchanged state (empty dictionary).
    """
    lr = hyperparams["lr"]  # Get the learning rate from hyperparameters.

    # TODO: Implement the standard SGD update rule
    new_params = params - lr*grad

    return new_params, state

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.1}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.05}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.01}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.001}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.0001}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.00001}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

# SGD Hyperparameter Analysis(5 Points)

For the SGD optimizer, we have defined the initial hyperparameters as follows:

    sgd_hyperparams = {"lr": 0.01}

In this section, you are required to test various learning rates and analyze how they affect the optimizer's performance. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the learning rate values you tested. Provide a clear list or table of these values.

###✅ Answer
| Learning Rate |
|--------------|
| 0.1          |
| 0.05         |
| 0.01         |
| 0.001        |
| 0.0001       |
| 0.00001      |


----
2. **Performance Analysis:**
   - How did different learning rates impact the convergence behavior of SGD?
   - Were there any specific learning rates that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested learning rates cause the optimizer to get stuck in local minima?

###✅ Answer
Based on my observations, different learning rates significantly impact the convergence behavior of Stochastic Gradient Descent (SGD).

A higher learning rate (like 0.1, 0.01) tends to cause instability, overshooting, or oscillations, especially in functions with sharp gradients or complex landscapes like Rosenbrock, Beale, and Ill-conditioned convex. Some functions, such as Rastrigin, show fluctuations, indicating that the step size is too large to settle into a minimum. In other cases, like Schwefel, Ackley, and Eggholder, the optimizer gets trapped in local minima, suggesting that high LR jumps too aggressively and fails to explore better solutions.

A lower learning rate (0.001, 0.0001) leads to extremely slow convergence. Many functions, such as Rastrigin, Schwefel, and Ackley, result in the optimizer getting stuck near the initial point, meaning the step size is too small to make meaningful progress. However, in well-conditioned convex landscapes, such as Beale and Ill-conditioned convex, this small LR allows steady progress and reaches the global minimum.

The results suggest that a moderate learning rate (e.g., 0.01 or 0.001) might provide a balance between speed and stability. A too-high LR risks instability, while a too-low LR may cause stagnation.

---

3. **Optimal Learning Rate:**
   - Which learning rate, among those tested, achieved the best balance between convergence speed and stability?
   - Explain why you consider this learning rate optimal based on your observations.

###✅ Answer
The optimal choice is in the range of 0.01 or 0.001. Because these values are small enough to reduce instability and overshooting. And they are large enough to allow meaningful updates without stagnation. Many optimization problems, particularly those with a mix of smooth and rugged landscapes, tend to perform well within this range.

---
4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying the learning rate. For instance, did higher learning rates lead to faster initial convergence at the risk of overshooting, or did lower learning rates provide more stable but slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

###✅ Answer
Based on my observations, adjusting the learning rate introduces clear trade-offs between convergence speed and stability:

- **Higher Learning Rate (0.1) – Fast Convergence, but Risky**

  - **Faster initial convergence**: In some cases, like 2D Beale and 2D Ill-conditioned convex, the optimizer moved rapidly but overshot.
  - **Overshooting & instability**: Functions like 2D Rosenbrock and 2D Ill-conditioned convex exhibited large oscillations, making it difficult for SGD to settle into an optimal solution.
  - **Fluctuations in rugged landscapes**: In highly non-convex functions like Rastrigin, the optimizer fluctuated, likely bouncing between local minima without settling.

- **Lower Learning Rate (0.0001) – Stable but Extremely Slow**

  - **Minimal movement in early iterations**: Many functions (e.g., 1D Rastrigin, 1D Schwefel, 2D Rosenbrock, and 2D Ackley) resulted in the optimizer being stuck near the initial point, suggesting that the step size was too small to make meaningful progress.
  - **Improved stability for smooth functions**: In convex landscapes (2D Beale, 2D Ill-conditioned convex), the optimizer successfully reached the global minimum without overshooting.
  - **Local minima trapping**: For rugged functions like Ackley and Eggholder, the small LR prevented the optimizer from escaping poor local solutions.

- **Emerging Patterns and Trends**
  - Higher LRs accelerate learning but at the cost of stability, often leading to oscillations or divergence.
  - Lower LRs ensure stability but can be too slow, requiring more iterations to reach an optimal solution.
  - Non-convex functions (e.g., Rastrigin, Eggholder, Ackley) are highly sensitive to LR. A high LR causes erratic behavior, while a low LR leads to stagnation.
  - Smooth convex functions (e.g., Beale, Ill-conditioned convex) benefit from a lower LR, as slow and steady updates help reach the global minimum.
---
5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate when using SGD in similar optimization tasks?

###✅ Answer
- Increase the rate slightly if the landscape is rugged or non-convex, but be cautious of overshooting.
- Use lower rates for smooth or convex functions to ensure convergence without instability.
- Consider learning rate schedules or adaptive methods to fine-tune the learning rate over time.
---
###⏩ Conclusion

| Learning Rate | Convergence Speed | Stability           | Behavior                                               |
|---------------|-------------------|---------------------|--------------------------------------------------------|
| 0.1           | Fast              | Unstable            | Overshooting, oscillations, fluctuations, and local minima trapping |
| 0.05          | Moderate          | Unstable            | Similar to 0.1, but slightly better stability in some cases, still prone to oscillations |
| 0.01          | Moderate          | More Stable         | May cause overshooting in some functions, but generally smooth convergence |
| 0.001         | Slow              | Stable              | Very slow convergence, but smoother behavior, may get stuck near initial point |
| 0.0001        | Very Slow         | Very Stable         | Slow, stuck near initial point for most functions, may reach global minima in smooth landscapes |
| 0.00001       | Extremely Slow    | Very Stable         | Almost no progress for most functions, may only converge in well-conditioned convex landscapes |

Make sure your report includes detailed observations, a comparative analysis, and a summary of all the learning rates tested. Your analysis should clearly illustrate how the choice of learning rate influences the performance of the SGD optimizer.


# SGD with Momentum (5 Points)

**Mathematical Formulation:**

The SGD with Momentum update is given by:

$$
v_t = \beta v_{t-1} + \eta \nabla f(w_t)
$$

$$
w_{t+1} = w_t - v_t
$$

where:  
- \\(w_t\\) is the parameter vector at iteration \\(t\\),  
- \\(v_t\\) is the velocity (accumulated momentum) at iteration \\(t\\),  
- \\(\eta\\) is the learning rate, and  
- \\(\beta\\) is the momentum coefficient (typically between 0 and 1).

**Explanation:**

SGD with Momentum introduces an additional term \\(v_t\\) that accumulates a running average of past gradients. This helps dampen oscillations and accelerates convergence, especially in scenarios where the gradient direction is consistent. However, careful tuning of both the learning rate \\(\eta\\) and the momentum coefficient \\(\beta\\) is necessary for optimal performance.


In [21]:
def momentum_update(params, grad, state, hyperparams, **kwargs):
    """
    SGD with Momentum update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store state (e.g., velocity). Initially, this can be empty.
      - hyperparams: Dictionary of hyperparameters. Must contain:
          "lr": learning rate, and
          "momentum": momentum coefficient.
      - **kwargs: Additional arguments (not used here).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary (contains the velocity).
    """
    lr = hyperparams["lr"]
    momentum = hyperparams["momentum"]

    if "v" not in state:
        # TODO: Initialize the velocity tensor with zeros, having the same shape as params
        state["v"] = torch.zeros_like(params)

    # TODO: Implement the momentum update rule: v_t = momentum * v_{t-1} + lr * grad
    state["v"] = momentum*state["v"] + lr*grad

    # TODO: Update parameters using the velocity term
    new_params = params - state["v"]

    return new_params, state

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.1,       # learning rate
    "momentum": 0.9    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.01,       # learning rate
    "momentum": 0.9    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.9    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.01,       # learning rate
    "momentum": 0.95    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.95    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.01,       # learning rate
    "momentum": 0.8    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.8    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)

# Momentum Hyperparameter Analysis(5 Points)

For the Momentum optimizer, the hyperparameters are defined as follows:




```python
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.9    # momentum coefficient
}
```



In this section, you are required to test various values for both the learning rate and the momentum coefficient, and analyze their impact on the optimizer's performance. Please answer the following questions based on your experiments:

1. Parameter Variation:
   - List all the learning rate values you tested.
   - List all the momentum coefficient values you tested.
   - Provide a clear list or table of these values along with any combinations you experimented with.

### ✅ Answer
| Learning Rate | Momentum Coefficient |
|--------------|----------------------|
| 0.1          | 0.9                  |
| 0.01         | 0.9                  |
| 0.001        | 0.9                  |
| 0.01         | 0.95                 |
| 0.001        | 0.95                 |
| 0.01         | 0.8                  |
| 0.001         | 0.8                  |


---
2. Performance Analysis:
   - How did changes in the learning rate affect the convergence behavior of the Momentum optimizer?
   - How did variations in the momentum coefficient influence the optimization process?
   - Were there specific combinations that led to faster convergence or instability (e.g., overshooting or slow progress)?

### ✅ Answer

- Higher Learning Rate (0.1):

    - Led to instability and overshooting in multiple cases.
    - The optimizer struggled to settle into a minimum, often oscillating.

- Moderate Learning Rate (0.01):

  - Improved stability compared to 0.1 but still showed overshooting in some cases.
  - Works well with certain momentum values but needs careful tuning.

- Lower Learning Rate (0.001):

  - Resulted in slow but stable convergence.
  - In some cases, reached the global minimum without overshooting.

- Higher Momentum (0.95):

  - Increased the risk of overshooting, particularly at 0.01 learning rate.
  - Helped in faster initial progress but sometimes made convergence unstable.

- Moderate Momentum (0.9):
  - Balanced speed and stability better than 0.95.
  - However, at a high learning rate (0.1), it still showed overshooting.

- Lower Momentum (0.8):

  - Reduced instability but slowed down convergence.
  - Worked better at 0.001 learning rate, but was not optimal for all cases.
---

3. Optimal Parameter Combination:
   - Which combination of learning rate and momentum coefficient achieved the best balance between convergence speed and stability?
   - Explain why you consider this combination optimal based on your observations.

### ✅ Answer

- Faster Convergence Compared to Lower Learning Rates

  - Unlike LR = 0.001, which was very stable but too slow, LR = 0.01 reached lower loss values faster.
  - It avoided getting stuck near the initial point, which was common with very low learning rates.

- More Stability Compared to Higher Learning Rates

 - Higher learning rates like 0.1 often caused overshooting and oscillations, making convergence unpredictable.
 - LR = 0.01 reduced these fluctuations while still maintaining relatively fast updates.

- Momentum = 0.9 Helped Maintain Smooth Descent

  - Momentum 0.9 provided enough acceleration to avoid slow progress, especially in functions with plateaus.
  - Unlike momentum 0.95, which frequently led to overshooting, 0.9 kept updates controlled.
  - Lower momentum values like 0.8 resulted in slower progress, especially in rugged landscapes.

---
4. Trade-offs and Observations:
   - Discuss any trade-offs observed when tuning these parameters. For example, did a higher momentum help accelerate convergence at the risk of overshooting, or did a lower momentum lead to more stable but slower updates?
   - Describe any trends or patterns that emerged from your experiments.

### ✅ Answer

- **Effect of Learning Rate**: A high learning rate led to rapid progress but often caused instability, overshooting, or divergence, especially in non-convex landscapes. While it helped escape shallow regions, it struggled to settle into an optimal solution. In contrast, a low learning rate ensured stable and precise updates but significantly slowed convergence, requiring more iterations to reach the minimum.

- **Effect of Momentum**: High momentum accelerated convergence by maintaining velocity, making it effective for escaping plateaus. However, it also increased the risk of overshooting and oscillations, especially with a high learning rate. Lower momentum provided more stability by reducing oscillations, but at the cost of slower convergence, making it less effective for navigating complex loss surfaces.

---
5. Recommendations:
   - Based on your experimental results, what recommendations would you give for selecting learning rate and momentum values when using momentum-based optimization methods in similar tasks?

### ✅ Answer
For most SGD-based optimization tasks, starting with LR = 0.01, Momentum = 0.9 is a strong baseline.
If instability is noticed, reduce LR slightly; if convergence is too slow, increase momentum slightly but avoid overshooting.

---

### ⏩ Conclusion

| Learning Rate | Momentum | Performance                                  |
|--------------|---------|----------------------------------------------- |
| **0.01**     | **0.9**  | ✅ Best balance between speed and stability   |
| **0.1**      | **0.9**  | ❌ Unstable, overshooting                     |
| **0.001**    | **0.9**  | ⚠️ Too slow but stable                        |
| **0.01**     | **0.95** | ❌ Overshooting                               |
| **0.01**     | **0.8**  | ⚠️ Slower than 0.9 but stable                 |


Ensure your report includes detailed observations, a comparative analysis, and a summary of all the hyperparameter values tested. Your analysis should clearly illustrate how the choice of learning rate and momentum coefficient influences the performance of the Momentum optimizer.


# Nesterov Accelerated Gradient (NAG)

**Mathematical Formulation:**

The Nesterov Accelerated Gradient update is given by:

$$
v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t)
$$

$$
w_{t+1} = w_t - \beta \, v_{t-1} - \eta \, \nabla f(w_t)
$$

where:
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( v_t \\) is the velocity (accumulated momentum) at iteration \\( t \\),
- \\( \eta \\) is the learning rate, and
- \\( \beta \\) is the momentum coefficient (typically a value between 0 and 1).

**Explanation:**

Nesterov Accelerated Gradient (NAG) is a variant of SGD with momentum that computes the gradient at a "lookahead" position. Instead of computing the gradient at the current parameter \\( w_t \\) only, NAG uses the accumulated momentum from previous iterations to “look ahead” before updating the parameters. The update can be interpreted as:

1. **Compute the momentum update:**  
   \\( v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t) \\)
2. **Update the parameters using the previous momentum:**  
   \\( w_{t+1} = w_t - \beta \, v_{t-1} - \eta \, \nabla f(w_t) \\)

This "lookahead" strategy often leads to faster convergence compared to standard momentum because it anticipates the effect of momentum and adjusts the update accordingly.

*Note:* In our implementation, we assume that we cannot recompute the gradient at the lookahead position (for simplicity) so we approximate NAG with the following update:

$$
v_{\text{prev}} = v_{t-1} \quad,\quad v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t)
$$

$$
w_{t+1} = w_t - \beta \, v_{\text{prev}} - \eta \, \nabla f(w_t)
$$

This formulation uses the previous momentum term directly for the "lookahead" component.


In [29]:
def nag_update(params, grad, state, hyperparams, **kwargs):
    """
    Nesterov Accelerated Gradient (NAG) update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params.
      - state: Dictionary to store the momentum (velocity). Initially empty.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate, and
            "momentum": momentum coefficient.
      - **kwargs: Additional arguments (ignored for NAG).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary containing the velocity.
    """
    lr = hyperparams["lr"]
    momentum = hyperparams["momentum"]

    # Initialize the velocity if not already in state.
    if "v" not in state:
        state["v"] = torch.zeros_like(params)

    # Save the previous velocity for the lookahead.
    v_prev = state["v"].clone()
    # Update the velocity.
    state["v"] = momentum * state["v"] + lr * grad
    # Update the parameters using the lookahead formula.
    new_params = params - momentum * v_prev - lr * grad
    return new_params, state

In [ ]:
# Define hyperparameters for Nesterov Accelerated Gradient.
nag_hyperparams = {
    "lr": 0.001,        # Learning rate
    "momentum": 0.9     # Momentum coefficient
}

# Run optimizations for all benchmark functions using NAG.
run_all_optimizations(update_func=nag_update, hyperparams=nag_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Nesterov Accelerated Gradient.
nag_hyperparams = {
    "lr": 0.01,        # Learning rate
    "momentum": 0.9     # Momentum coefficient
}

# Run optimizations for all benchmark functions using NAG.
run_all_optimizations(update_func=nag_update, hyperparams=nag_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Nesterov Accelerated Gradient.
nag_hyperparams = {
    "lr": 0.01,        # Learning rate
    "momentum": 0.95     # Momentum coefficient
}

# Run optimizations for all benchmark functions using NAG.
run_all_optimizations(update_func=nag_update, hyperparams=nag_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Nesterov Accelerated Gradient.
nag_hyperparams = {
    "lr": 0.001,        # Learning rate
    "momentum": 0.95     # Momentum coefficient
}

# Run optimizations for all benchmark functions using NAG.
run_all_optimizations(update_func=nag_update, hyperparams=nag_hyperparams, epochs=100)

# NAG Hyperparameter Analysis(5 Points)

For the Nesterov Accelerated Gradient (NAG) optimizer, we have defined the initial hyperparameters as follows:

    nag_hyperparams = {
        "lr": 0.001,        # Learning rate
        "momentum": 0.9     # Momentum coefficient
    }

In this section, you are required to test various learning rates and momentum values to analyze how they affect the performance of NAG. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the learning rate and momentum values you tested. Provide a clear list or table of these values.

### ✅ Answer
| Learning Rate | Momentum |
|--------------|----------|
| 0.001        | 0.9      |
| 0.01         | 0.9      |
| 0.01         | 0.95     |
| 0.001        | 0.95     |

---
2. **Performance Analysis:**
   - How did varying the learning rate and momentum coefficient impact the convergence behavior of NAG?
   - Were there any specific combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested parameter combinations cause the optimizer to get trapped in local minima?

### ✅ Answer

#### Impact on Convergence Behavior
- Lower Learning Rate (0.001) with Momentum (0.9 & 0.95):
  - Often resulted in local minima or slow convergence.
  - Example: On the 1D Rastrigin and 2D Rastrigin functions, it found local minima near the initial point.
  - On the 1D Ill-conditioned convex and 2D Rosenbrock functions, it successfully reached the global minimum, but likely at a slow pace.

- Higher Learning Rate (0.01) with Momentum (0.9 & 0.95):
  - More aggressive updates led to overshooting in some cases.
  - Example: In the 2D Rosenbrock function, increasing the learning rate from 0.001 to 0.01 caused overshoot, meaning it oscillated around the optimal solution rather than converging smoothly.
  - In the 1D Schwefel function, it showed a smoother learning curve at 0.01, momentum 0.9, but at 0.01, momentum 0.95, it got stuck near the initial point.

- Unstable Behavior & Overshooting
  - 2D Rosenbrock function with (lr=0.01, momentum=0.9 or 0.95)
  - Caused overshooting, indicating that momentum was too high relative to the learning rate.
  - 1D Schwefel function (lr=0.01, momentum=0.95)

- Low learning rate (0.001) → slow convergence but stable results.
- High learning rate (0.01) → potential overshooting.
- Momentum 0.95 → risk of optimizer getting stuck or overshooting.
- Momentum 0.9 was generally more stable but sometimes led to slow convergence.

---
3. **Optimal Parameters:**
   - Which combination of learning rate and momentum achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer
For a good trade-off between speed and stability, use lr = 0.01, momentum = 0.9.

The combination of **learning rate = 0.01 and momentum = 0.9** provides the best balance between convergence speed and stability because it accelerates learning without excessive overshooting or getting stuck in local minima. Unlike **lr = 0.001**, which often results in slow convergence and local minima trapping, **lr = 0.01** allows the optimizer to escape these traps while maintaining efficiency. Additionally, **momentum = 0.9** helps smooth updates and accelerate convergence without the instability observed at **momentum = 0.95**, which sometimes caused overshooting (e.g., in 2D Rosenbrock) or stagnation (e.g., in 1D Schwefel). This setting consistently led to smooth, decreasing learning curves across various functions, making it the most reliable choice.

---
4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying both the learning rate and momentum. For example, did higher momentum values speed up convergence but risk overshooting, or did lower momentum values result in more stable but slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer
- Higher learning rates helped escape local minima but needed controlled momentum to prevent overshooting.
- Lower learning rates ensured stability but resulted in slow progress or local minima trapping.
- Higher Momentum increased speed but risked instability.
- Lower Momentum would likely slow down convergence further.
---
5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate and momentum when using NAG in similar optimization tasks?

### ✅ Answer
For optimizing with Nesterov Accelerated Gradient (NAG), start with **lr = 0.01, momentum = 0.9**, as it offers a solid balance between speed and stability. If convergence is too slow, slightly increase **lr** or **momentum**, but be cautious of overshooting. For highly sensitive functions prone to instability, reduce **lr** while keeping **momentum at 0.9** to maintain steady progress. If loss curves oscillate, momentum is likely too high and should be lowered. Conversely, if the optimizer gets stuck in local minima, increasing momentum can help escape. Always monitor the loss curve trends and fine-tune parameters based on stability and convergence speed.

---

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the NAG optimizer.


# Adagrad (Adaptive Gradient)

**Mathematical Formulation:**

Adagrad adapts the learning rate for each parameter individually by accumulating the squared gradients. Its update equations are:

$$
G_t = G_{t-1} + \left(\nabla f(w_t)\right)^2
$$

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{G_t} + \epsilon} \, \nabla f(w_t)
$$

where:  
- \\(w_t\\) is the parameter vector at iteration \\(t\\),  
- \\(G_t\\) is the accumulation of squared gradients (applied elementwise),  
- \\(\eta\\) is the learning rate, and  
- \\(\epsilon\\) is a small constant added for numerical stability.

**Explanation:**

Adagrad adjusts the learning rate based on the history of gradients. Parameters with high gradients accumulate a larger \\(G_t\\) and thus receive a smaller effective learning rate, while parameters with small or sparse gradients receive larger updates. This makes Adagrad particularly useful for problems with sparse features, but note that its learning rate can decay too aggressively over time.


In [38]:
def adagrad_update(params, grad, state, hyperparams, **kwargs):
    """
    Adagrad update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store state (e.g., accumulated squared gradients).
      - hyperparams: Dictionary of hyperparameters. Must contain:
          "lr": learning rate, and
          "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Adagrad).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    eps = hyperparams["eps"]

    if "G" not in state:
        state["G"] = torch.zeros_like(params)

    # Accumulate squared gradients.
    state["G"] += grad**2

    # Perform the Adagrad update.
    new_params = params - lr * grad / (torch.sqrt(state["G"]) + eps)
    return new_params, state

In [ ]:
# Define hyperparameters for Adagrad.
adagrad_hyperparams = {
    "lr": 0.01,   # Learning rate
    "eps": 1e-8   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Adagrad.
run_all_optimizations(update_func=adagrad_update, hyperparams=adagrad_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Adagrad.
adagrad_hyperparams = {
    "lr": 0.01,   # Learning rate
    "eps": 1e-4   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Adagrad.
run_all_optimizations(update_func=adagrad_update, hyperparams=adagrad_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Adagrad.
adagrad_hyperparams = {
    "lr": 0.5,   # Learning rate
    "eps": 1e-2   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Adagrad.
run_all_optimizations(update_func=adagrad_update, hyperparams=adagrad_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for Adagrad.
adagrad_hyperparams = {
    "lr": 0.5,   # Learning rate
    "eps": 1e-4   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Adagrad.
run_all_optimizations(update_func=adagrad_update, hyperparams=adagrad_hyperparams, epochs=100)

# Adagrad Hyperparameter Analysis(5 Points)

For the Adagrad optimizer, we have defined the initial hyperparameters as follows:

    adagrad_hyperparams = {
        "lr": 0.01,   # Learning rate
        "eps": 1e-8   # Small constant for numerical stability
    }

In this section, you are required to test various learning rates and epsilon values to analyze how they affect the performance of Adagrad. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the learning rate and epsilon values you tested. Provide a clear list or table of these values.

### ✅ Answer

| Learning Rate | Epsilon |
|--------------|----------|
| 0.01         | 1e-8     |
| 0.5          | 1e-8     |
| 0.01         | 1e-4     |
| 0.5          | 1e-4     |

---
2. **Performance Analysis:**
   - How did different learning rates and epsilon values impact the convergence behavior of Adagrad?
   - Were there any specific parameter combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested combinations cause the optimizer to get stuck in local minima?

### ✅ Answer

- Lower Learning Rate (0.01) → Slow Convergence and Local Minima Trapping
- Higher Learning Rate (0.5) → Faster Convergence and Improved Escape from Local Minima
- Increasing lr to 0.5 allowed the optimizer to escape local minima in 1D and 2D

- Changing epsilon from 1e-8 to 1e-4 had minimal effect on convergence.
- The primary driver of convergence behavior was the learning rate rather than epsilon.
- Low lr in all cases resulted in very slow updates, often causing the optimizer to barely move.
- High lr improved convergence but did not always reach a precise global minimum.

---
3. **Optimal Parameters:**
   - Which combination of learning rate and epsilon achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer

- Best practice: Learning Rate = 0.5, Epsilon = 1e-8 or 1e-4

- **Faster Convergence Without Getting Stuck**: Unlike lr = 0.01, which often resulted in almost no movement, lr = 0.5 allowed the optimizer to escape local minima and make meaningful progress across functions like 1D Rastrigin, 2D Rastrigin, and 2D Rosenbrock.
- **Stable Updates Without Overshooting**: Unlike momentum-based optimizers, Adagrad naturally adapts the learning rate, preventing excessive oscillations.
- **Better Exploration of Complex Functions**: The optimizer successfully reduced loss in challenging landscapes like 2D Rosenbrock and Ill-conditioned convex, where lower learning rates had trouble escaping local traps.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying the learning rate and epsilon values. For instance, did a lower epsilon improve stability at the cost of slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer
- Low learning rate (0.01) consistently caused slow convergence and local minima trapping.
- Higher learning rate (0.5) enabled fast convergence, but final solutions were sometimes near-optimal rather than exact.
- Changing epsilon had negligible effects compared to adjusting the learning rate.
- Adagrad’s adaptive nature prevented overshooting, so tuning learning rate was more important than tuning epsilon.

---

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate and epsilon when using Adagrad in similar optimization tasks?

### ✅ Answer

For most optimization tasks, set lr = 0.5 and epsilon = 1e-8 as a strong default. Adjust the learning rate only if convergence is too slow (increase lr) or unstable (slightly decrease lr). There is usually no need to modify epsilon, as it has little impact on Adagrad’s overall performance.

---
Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adagrad optimizer.


# RMSprop (Root Mean Square Propagation)

**Mathematical Formulation:**

RMSprop adapts the learning rate for each parameter by maintaining an exponentially decaying average of squared gradients. The update equations are:

$$
E[g^2]_t = \beta \, E[g^2]_{t-1} + (1 - \beta) \, \left(\nabla f(w_t)\right)^2
$$

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{E[g^2]_t} + \epsilon} \, \nabla f(w_t)
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(E[g^2]_t\\) is the exponentially decaying average of squared gradients,
- \\(\eta\\) is the learning rate,
- \\(\beta\\) is the decay rate (typically around 0.9), and
- \\(\epsilon\\) is a small constant added for numerical stability.

**Explanation:**

RMSprop is designed to overcome the aggressive decay of the learning rate seen in Adagrad by using an exponential decay factor \\(\beta\\) to forget old gradients. This helps to maintain a more stable and adaptive learning rate during training. RMSprop works well in practice on non-stationary problems and deep networks.


In [48]:
def rmsprop_update(params, grad, state, hyperparams, **kwargs):
    """
    RMSprop update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to parameters.
      - state: Dictionary to store state (e.g., running average of squared gradients).
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta": decay rate, and
            "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for RMSprop).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    beta = hyperparams["beta"]
    eps = hyperparams["eps"]

    if "E" not in state:
        state["E"] = torch.zeros_like(params)

    # Update the running average of squared gradients.
    state["E"] = beta * state["E"] + (1 - beta) * grad**2

    # Compute the RMSprop update.
    new_params = params - lr * grad / (torch.sqrt(state["E"]) + eps)
    return new_params, state

In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.001,    # Learning rate
    "beta": 0.9,    # Decay rate
    "eps": 1e-3     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.1,    # Learning rate
    "beta": 0.9,    # Decay rate
    "eps": 1e-8     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.001,    # Learning rate
    "beta": 0.95,    # Decay rate
    "eps": 1e-8     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.1,    # Learning rate
    "beta": 0.95,    # Decay rate
    "eps": 1e-8     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.001,    # Learning rate
    "beta": 0.9,    # Decay rate
    "eps": 1e-8     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)

# RMSprop Hyperparameter Analysis (5 Points)

## 1. Parameter Variation:
| Learning Rate | Beta  | Epsilon |
|--------------|-------|----------|
| 0.001        | 0.9   | 1e-3     |
| 0.1          | 0.9   | 1e-8     |
| 0.001        | 0.95  | 1e-8     |
| 0.1          | 0.95  | 1e-8     |
| 0.001        | 0.9   | 1e-8     |

---


## 2. Performance Analysis:

- Effect of Learning Rate (lr):
  - Lower Learning Rate (0.001): Provided stability but resulted in very slow convergence.
  - Struggled to escape local minima, especially in complex loss landscapes.
  - More suitable for highly sensitive tasks requiring careful, gradual updates.

- Higher Learning Rate (0.1):

  - Faster convergence across most test cases, significantly outperforming lr = 0.001.
  - Showed occasional instability and oscillations, especially when paired with lower beta values.
  - Worked best when combined with β = 0.95, which smoothed out excessive fluctuations.

- Effect of Beta:
 - Lower Beta (0.9): Allowed for faster adaptation to recent gradients but could introduce more noise.
   - Led to marginally better responsiveness, making updates slightly quicker.
   - In some cases, introduced small oscillations when paired with a high learning rate.

 - Higher Beta (0.95):
   - Retained historical gradient information for a longer period, smoothing out updates.
   - Helped reduce instability when using a high learning rate (0.1), leading to better convergence.
   - Slightly slower updates compared to β = 0.9, but this trade-off helped avoid erratic jumps.

- Effect of Epsilon:
  - Higher Epsilon (1e-3): Improved numerical stability but slightly slowed down learning.
  - Best suited for situations where gradients are extremely small or erratic.

  - Lower Epsilon (1e-8): Allowed for more aggressive updates, making learning more responsive.
  - More effective when paired with a higher learning rate (0.1).
  - Risked slight instability in certain cases but was generally preferred for smoother optimizations.

---

## 3. Optimal Parameter Combination:

✅ Best Settings: lr = 0.1, β = 0.95, ε = 1e-8

🔹 Why This is Optimal:
- Achieved the best balance between convergence speed and stability.
- Higher β (0.95) controlled oscillations, preventing overshooting while maintaining momentum.
- Lower ε (1e-8) allowed efficient learning, without unnecessary dampening.

---

## 4. Trade-offs and Observations:

| Hyperparameter | Effect |
|---------------|--------|
| **Low lr (0.001)** | Stable but very slow, struggles to escape local minima. |
| **High lr (0.1)** | Fast convergence but needs β = 0.95 for stability. |
| **Low β (0.9)** | Faster adaptation but slightly noisier updates. |
| **High β (0.95)** | Smoother updates, more stable but slightly slower. |
| **Low ε (1e-8)** | Works best for learning efficiency, may introduce slight instability. |
| **High ε (1e-3)** | Ensures stability, but slows down convergence. |


---

## 5. Recommendations:

- Start with lr = 0.1, β = 0.95, and ε = 1e-8 for a strong balance of speed and stability.
- Lower lr to 0.001 if dealing with highly sensitive gradients or extreme noise.
- Use β = 0.9 for slightly faster updates, but prefer β = 0.95 for smoother learning.
- Keep ε at 1e-8 unless encountering numerical instability, in which case 1e-3 can be used.

# Adam (Adaptive Moment Estimation)(10 Points)

**Mathematical Formulation:**

Adam computes two exponential moving averages of the gradients:

- **First moment (mean):**

$$
m_t = \beta_1 m_{t-1} + (1 - \beta_1) \nabla f(w_t)
$$

- **Second moment (uncentered variance):**

$$
v_t = \beta_2 v_{t-1} + (1 - \beta_2) \left(\nabla f(w_t)\right)^2
$$

Bias corrections are then applied:

$$
\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$

Finally, the parameter update is given by:

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \, \hat{m}_t
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate,
- \\(\beta_1\\) and \\(\beta_2\\) are the exponential decay rates for the moment estimates, and
- \\(\epsilon\\) is a small constant to avoid division by zero.

**Explanation:**

Adam combines ideas from momentum and RMSprop. By maintaining an exponentially decaying average of past gradients (first moment) and past squared gradients (second moment) along with bias correction, Adam adapts the learning rate for each parameter. This makes it especially suitable for problems with noisy or sparse gradients, as it works well "out of the box" with minimal tuning.


# RMSprop Hyperparameter Analysis(5 Points)

For the RMSprop optimizer, we have defined the initial hyperparameters as follows:

    rmsprop_hyperparams = {
        "lr": 0.001,    # Learning rate
        "beta": 0.9,    # Decay rate
        "eps": 1e-8     # Small constant for numerical stability
    }

In this section, you are required to test various learning rates, beta values, and epsilon values to analyze how they affect the performance of RMSprop. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the learning rate, beta, and epsilon values you tested. Provide a clear list or table of these values.

2. **Performance Analysis:**
   - How did different combinations of learning rate, beta, and epsilon impact the convergence behavior of RMSprop?
   - Were there any specific combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested combinations cause the optimizer to get stuck in local minima?

3. **Optimal Parameters:**
   - Which combination of parameters achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying these hyperparameters. For example, did a higher beta improve stability at the cost of slower adaptation?
   - Describe any patterns or trends that emerged from your experiments.

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate, beta, and epsilon when using RMSprop in similar optimization tasks?

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the RMSprop optimizer.


In [51]:
def adam_update(params, grad, state, hyperparams, **kwargs):
    """
    Adam update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params.
      - state: Dictionary to store moving averages for first and second moments.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta1": decay rate for the first moment,
            "beta2": decay rate for the second moment,
            "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Adam).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    beta1 = hyperparams["beta1"]
    beta2 = hyperparams["beta2"]
    eps = hyperparams["eps"]

    if "m" not in state:
        # TODO: Initialize first moment (m), second moment (v), and timestep (t)
        state["m"] = torch.zeros_like(params)
        state["v"] = torch.zeros_like(params)
        state["t"] = 0

    # TODO: Update timestep
    state["t"] += 1

    # TODO: Compute biased first moment estimate (m) and second moment estimate (v)
    state["m"] = beta1*state["m"] + (1 - beta1)*grad
    state["v"] = beta2*state["v"] + (1 - beta2)*(grad**2)

    # TODO: Compute bias-corrected first moment estimate (m_hat) and second moment estimate (v_hat)
    m_hat = state["m"] / (1 - beta1**state["t"])
    v_hat = state["v"] / (1 - beta2**state["t"])

    # TODO: Update parameters using Adam's rule
    new_params = params - lr*(m_hat / (torch.sqrt(v_hat) + eps))

    return new_params, state

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.85,     # Decay rate for first moment
    "beta2": 0.98,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.01,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.85,     # Decay rate for first moment
    "beta2": 0.98,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.1,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.1,      # Learning rate
    "beta1": 0.85,     # Decay rate for first moment
    "beta2": 0.98,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)

# Adam Hyperparameter Analysis(5 Points)

For the Adam optimizer, we have defined the initial hyperparameters as follows:

    adam_hyperparams = {
        "lr": 0.001,      # Learning rate
        "beta1": 0.9,     # Decay rate for first moment
        "beta2": 0.999,   # Decay rate for second moment
        "eps": 1e-8       # Smoothing constant
    }

In this section, you are required to test various combinations of the learning rate, beta1, beta2, and epsilon values to analyze how they affect the performance of Adam. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the values you tested for learning rate, beta1, beta2, and epsilon. Provide a clear list or table of these parameter combinations.

### ✅ Answer

| Learning Rate (lr) | Beta1 (β₁) | Beta2 (β₂) |
|-------------------|------------|------------|
| **0.001**        | **0.9**     | **0.999**  |
| **0.001**        | **0.85**    | **0.98**   |
| **0.01**         | **0.9**     | **0.999**  |
| **0.01**         | **0.85**    | **0.98**   |
| **0.1**          | **0.9**     | **0.999**  |
| **0.1**          | **0.85**    | **0.98**   |

2. **Performance Analysis:**
   - How did different parameter combinations impact the convergence behavior of Adam?
   - Were there any specific combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested parameter combinations cause the optimizer to get stuck in local minima?

### ✅ Answer

- Effect of Learning Rate:
  - Lower LR (0.001): Very stable, but optimization was slow and sometimes got stuck in local minima. Performed better with lower decay rates (β₁ = 0.85, β₂ = 0.98) due to increased adaptation.

 - Moderate LR (0.01): Balanced speed and stability, leading to better convergence. Best results when paired with β₁ = 0.9 and β₂ = 0.999, as it maintained sufficient momentum while adapting quickly.

  - Higher LR (0.1): Fastest convergence, but increased risk of instability and oscillations, especially with β₁ = 0.85.

- Effect of Beta1:
 - Lower β₁ (0.85): Allowed faster adaptation to new gradients, improving performance in dynamic loss landscapes. Increased sensitivity to noise, making updates more aggressive.


 - Higher β₁ (0.9): Retained past gradient information longer, resulting in smoother updates. Preferred for higher learning rates (0.1) as it reduced instability.

- Effect of Beta2:
  - Lower β₂ (0.98): Allowed quicker adaptation to gradient updates, improving responsiveness. Performed better at lower learning rates (0.001, 0.01) but introduced oscillations at 0.1 LR.

 - Higher β₂ (0.999): Provided greater stability by averaging squared gradients over a longer window. Slowed down adaptation, sometimes preventing rapid convergence.
Best suited for higher learning rates (0.1) to counteract instability.

---

3. **Optimal Parameters:**
   - Which combination of parameters achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer

- Best Settings: lr = 0.01, β₁ = 0.9, β₂ = 0.999

 - Balanced stability and adaptability, leading to consistent convergence.
 - Lower β₂ (0.98) allowed faster updates, but β₂ = 0.999 ensured smoother learning.
 - Avoided instability that occurred at lr = 0.1 while still being faster than lr = 0.001.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying these hyperparameters. For example, did a lower epsilon or higher beta values improve stability at the cost of slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

### Answer

| Hyperparameter | Effect |
|---------------|--------|
| **Low lr (0.001)** | Very stable but slow; struggles to escape local minima. |
| **Moderate lr (0.01)** | Best balance of speed and stability. |
| **High lr (0.1)** | Fast convergence but risks instability without high β values. |
| **Low β₁ (0.85)** | Faster adaptation but noisier updates. |
| **High β₁ (0.9)** | Smoother updates, reducing instability at high LR. |
| **Low β₂ (0.98)** | Faster adaptation but risked oscillations. |
| **High β₂ (0.999)** | More stable but slightly slower optimization. |

---

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate, beta1, beta2, and epsilon when using Adam in similar optimization tasks?

### ✅ Answer

- Start with lr = 0.01, β₁ = 0.9, β₂ = 0.999 for a strong balance of convergence speed and stability.
- If training is slow, reduce β₂ to 0.98 for quicker adaptation.
- If instability occurs, increase β₂ to 0.999 and reduce lr slightly.
- For highly noisy gradients, prefer β₁ = 0.9 to smooth out updates.
---
Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adam optimizer.


# Newton’s Method(5 Points)

**Mathematical Formulation:**

Newton’s method uses second-order information (the Hessian) to update the parameters. The update rule is given by:

$$
w_{t+1} = w_t - H(w_t)^{-1} \nabla f(w_t)
$$

where:
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( \nabla f(w_t) \\) is the gradient of the loss function at \\( w_t \\), and
- \\( H(w_t) \\) is the Hessian matrix (i.e., the matrix of second derivatives) evaluated at \\( w_t \\).

For numerical stability (and to handle cases when the Hessian is nearly singular), a small constant \\( \epsilon \\) is added to the diagonal of the Hessian:

$$
w_{t+1} = w_t - \left( H(w_t) + \epsilon I \right)^{-1} \nabla f(w_t)
$$

**Explanation:**

Newton’s method can converge very rapidly (quadratically) when the loss function is well-behaved and when you are close to the optimum. However, computing the full Hessian (and its inverse) can be computationally expensive for high-dimensional problems. Because of these challenges, Newton’s method (or its quasi-Newton variants like L-BFGS) is typically used for smaller models or as part of a hybrid optimization strategy.

*Note:* In our implementation, we assume the loss function is defined using PyTorch, the parameters are a torch tensor of shape \\( (n,) \\) for 1D functions or \\( (2,) \\) for 2D functions, and the Hessian is computed via PyTorch’s automatic differentiation.


In [52]:
def newton_update(params, grad, state, hyperparams, hessian):
    """
    Newton's Method update function.

    Parameters:
      - params: Current parameter values (torch tensor, shape (n,) for 1D or (2,) for 2D).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state (not used in basic Newton's method).
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "eps": A small constant for numerical stability.
      - hessian: Hessian matrix of the loss function (torch tensor of shape (n, n)).

    Returns:
      - new_params: Updated parameter values.
      - state: Unchanged state dictionary.
    """
    eps = hyperparams["eps"]

    # TODO: Create an identity matrix of the same size as params
    I = torch.eye(params.shape[0])

    # TODO: Regularize the Hessian for numerical stability
    H_reg = hessian + eps * I

    # TODO: Compute the inverse of the regularized Hessian
    H_inv = torch.inverse(H_reg)

    # TODO: Compute the Newton update step: new_params = params - H_inv @ grad
    new_params = params - (H_inv) @ grad

    return new_params, state

In [ ]:
# Define hyperparameters for Newton's method.
newton_hyperparams = {
    "eps": 1e-4   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Newton's method.
# Note: Newton's method is a second-order method, so our general optimize function should be called with second_order=True.
run_all_optimizations(update_func=newton_update, hyperparams=newton_hyperparams, epochs=100, second_order=True)

# Newton's Method Hyperparameter Analysis(5 Points)

For Newton's Method, we have defined the initial hyperparameters as follows:

    newton_hyperparams = {
        "eps": 1e-4   # Small constant for numerical stability
    }

In this section, you are required to test different values for the epsilon parameter to analyze its effect on the performance of Newton's Method. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the epsilon values you tested. Provide a clear list or table of these values.

### ✅ Answer

Newton’s method was tested with an epsilon value of 1e-4, chosen to ensure numerical stability while maintaining effective convergence.

---

2. **Performance Analysis:**
   - How did different epsilon values impact the convergence behavior of Newton's Method?
   - Were there any specific epsilon values that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested epsilon values cause the optimizer to get stuck in local minima or result in numerical instability?

### ✅ Answer

Newton’s method demonstrated rapid convergence on well-behaved functions, particularly those with smooth, convex surfaces. It performed exceptionally well on functions Ill-Conditioned, Rosenbrock, and 2D Ill-Conditioned, where second-order updates efficiently guided optimization. However, the method exhibited overshooting or converging along suboptimal paths in several tests due to its aggressive step size. Additionally, it frequently became trapped in local minima.

---

3. **Optimal Parameter:**
   - Which epsilon value, among those tested, achieved the best balance between convergence speed and stability?
   - Explain why you consider this epsilon value optimal based on your observations.

### ✅ Answer

The optimal choice of epsilon (ε = 1e-4) provided sufficient numerical stability, preventing division errors in Hessian inversion while ensuring effective optimization. This small value helped maintain convergence reliability without significantly impairing the step size.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying epsilon. For example, did a smaller epsilon provide more precise curvature estimates at the cost of slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer
Newton’s method offers extremely fast and accurate convergence near global minima, making it ideal for well-conditioned or convex problems. However, its reliance on second-order derivatives makes it highly sensitive to initial values and computationally expensive due to the need for Hessian computation and inversion. While it efficiently optimizes smooth landscapes, it struggles with non-convex or ill-conditioned problems, where it frequently gets stuck in local minima or takes excessively large steps, leading to instability.

---

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the epsilon parameter when using Newton's Method in similar optimization tasks?

### ✅ Answer

Newton’s method excels in optimizing well-conditioned, convex, and low-dimensional problems, where leveraging second-order information significantly enhances performance. Keeping epsilon small (1e-4) helps maintain numerical stability while preserving an effective convergence rate.

---

Ensure your report includes detailed observations, a comparative analysis, and a summary of all epsilon values tested. Your analysis should clearly illustrate how the choice of this hyperparameter influences the performance of Newton's Method.


# L-BFGS (Limited-memory BFGS)

**Mathematical Formulation:**

L-BFGS is a quasi-Newton method that approximates the inverse Hessian matrix using a limited history of parameter and gradient differences. It avoids storing the full Hessian, making it memory efficient for large-scale problems.

Suppose at iteration \\(t\\) we have stored \\(m\\) pairs \\((s_i, y_i)\\) for \\(i = t-m, \ldots, t-1\\), where

$$
s_i = w_{i+1} - w_i \quad \text{and} \quad y_i = \nabla f(w_{i+1}) - \nabla f(w_i).
$$

The two-loop recursion for computing the approximate inverse Hessian times the gradient \\(q = \nabla f(w_t)\\) is as follows:

1. **First Loop (backward pass):**

   For \\(i = t-1, t-2, \ldots, t-m\\):

   $$
   \rho_i = \frac{1}{y_i^T s_i} \quad,\quad \alpha_i = \rho_i \, s_i^T q
   $$

   Update:

   $$
   q \leftarrow q - \alpha_i \, y_i
   $$

2. **Scaling the Initial Hessian:**

   Use an initial Hessian approximation:

   $$
   H_0 = \gamma I \quad \text{with} \quad \gamma = \frac{s_{t-1}^T y_{t-1}}{y_{t-1}^T y_{t-1}}
   $$

   Set:

   $$
   r = H_0 q
   $$

3. **Second Loop (forward pass):**

   For \\(i = t-m, \ldots, t-1\\):

   $$
   \beta_i = \rho_i \, y_i^T r
   $$

   Update:

   $$
   r \leftarrow r + s_i (\alpha_i - \beta_i)
   $$

The parameter update is then:

$$
w_{t+1} = w_t - r.
$$

**Explanation:**

L-BFGS efficiently approximates the Newton direction without computing or storing the full Hessian matrix. By keeping a limited memory of recent changes in parameters and gradients, it builds a useful approximation of the inverse Hessian that can lead to fast convergence on large-scale problems. The hyperparameter \\(m\\) controls the memory size (i.e., how many past updates are retained), and the initial scaling factor \\(\gamma\\) helps initialize the inverse Hessian approximation.

*Note:* In our implementation, we maintain lists of differences \\(s_i\\) and \\(y_i\\), and use the two-loop recursion to compute the search direction.


In [56]:
def lbfgs_update(params, grad, state, hyperparams, **kwargs):
    """
    L-BFGS update function (simplified version).

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state.
          Expected keys:
            "S": list of previous parameter differences.
            "Y": list of previous gradient differences.
            "prev_params": previous parameters (torch tensor).
            "prev_grad": previous gradient (torch tensor).
      - hyperparams: Dictionary of hyperparameters.
          Must contain:
            "lr": initial learning rate scaling factor,
            "m": memory size (maximum number of corrections to store).
      - **kwargs: Additional arguments (ignored for L-BFGS).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    m_memory = hyperparams["m"]

    # If no previous state exists, do a simple gradient descent step.
    if "prev_params" not in state:
        new_params = params - lr * grad
        state["prev_params"] = params.clone().detach()
        state["prev_grad"] = grad.clone().detach()
        state["S"] = []
        state["Y"] = []
        return new_params, state

    # Compute parameter difference s and gradient difference y.
    s = params - state["prev_params"]
    y = grad - state["prev_grad"]

    # Update history lists.
    if len(state["S"]) >= m_memory:
        state["S"].pop(0)
        state["Y"].pop(0)
    state["S"].append(s.clone().detach())
    state["Y"].append(y.clone().detach())

    # Update previous parameters and gradient.
    state["prev_params"] = params.clone().detach()
    state["prev_grad"] = grad.clone().detach()

    # Two-loop recursion.
    q = grad.clone().detach()
    alpha = []
    # Loop from the most recent history to the oldest.
    for s_i, y_i in zip(reversed(state["S"]), reversed(state["Y"])):
        rho_i = 1.0 / (torch.dot(y_i.view(-1), s_i.view(-1)) + 1e-10)
        a_i = rho_i * torch.dot(s_i.view(-1), q.view(-1))
        alpha.append(a_i)
        q = q - a_i * y_i

    # Scaling for initial Hessian approximation.
    if len(state["Y"]) > 0:
        last_y = state["Y"][-1]
        last_s = state["S"][-1]
        gamma = torch.dot(last_y.view(-1), last_s.view(-1)) / (torch.dot(last_y.view(-1), last_y.view(-1)) + 1e-10)
    else:
        gamma = 1.0

    r = gamma * q
    # Forward loop: from oldest to newest.
    for s_i, y_i, a_i in zip(state["S"], state["Y"], reversed(alpha)):
        rho_i = 1.0 / (torch.dot(y_i.view(-1), s_i.view(-1)) + 1e-10)
        beta = rho_i * torch.dot(y_i.view(-1), r.view(-1))
        r = r + s_i * (a_i - beta)

    # Update parameters: move in the direction r.
    new_params = params - r
    return new_params, state

In [ ]:
# Define hyperparameters for L-BFGS.
lbfgs_hyperparams = {
    "lr": 0.01,   # initial learning rate scaling factor
    "m": 10       # memory size (maximum corrections to store)
}

# Run optimizations for all benchmark functions using L-BFGS.
run_all_optimizations(update_func=lbfgs_update, hyperparams=lbfgs_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for L-BFGS.
lbfgs_hyperparams = {
    "lr": 0.01,   # initial learning rate scaling factor
    "m": 15       # memory size (maximum corrections to store)
}

# Run optimizations for all benchmark functions using L-BFGS.
run_all_optimizations(update_func=lbfgs_update, hyperparams=lbfgs_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for L-BFGS.
lbfgs_hyperparams = {
    "lr": 0.001,   # initial learning rate scaling factor
    "m": 10       # memory size (maximum corrections to store)
}

# Run optimizations for all benchmark functions using L-BFGS.
run_all_optimizations(update_func=lbfgs_update, hyperparams=lbfgs_hyperparams, epochs=100)

In [ ]:
# Define hyperparameters for L-BFGS.
lbfgs_hyperparams = {
    "lr": 0.001,   # initial learning rate scaling factor
    "m": 15       # memory size (maximum corrections to store)
}

# Run optimizations for all benchmark functions using L-BFGS.
run_all_optimizations(update_func=lbfgs_update, hyperparams=lbfgs_hyperparams, epochs=100)

# L-BFGS Hyperparameter Analysis

For the L-BFGS optimizer, we have defined the initial hyperparameters as follows:

    lbfgs_hyperparams = {
        "lr": 0.01,   # initial learning rate scaling factor
        "m": 10       # memory size (maximum corrections to store)
    }

In this section, you are required to test various values for the learning rate and memory size to analyze how they affect the performance of L-BFGS. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the learning rate and memory size (m) values you tested. Provide a clear list or table of these parameter combinations.

### ✅ Answer

| Learning Rate (LR) | Memory Size (m) |
|--------------------|-----------------|
| 0.001             | 15              |
| 0.001             | 10              |
| 0.01              | 15              |
| 0.01              | 10              |


---

2. **Performance Analysis:**
   - How did different combinations of learning rate and memory size impact the convergence behavior of L-BFGS?
   - Were there any specific parameter combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested combinations cause the optimizer to get trapped in local minima or exhibit other issues?

### ✅ Answer

- Effect of Learning Rate (LR):
  - A lower LR (0.001) resulted in stable but very slow convergence, often struggling to escape local minima.
  - A higher LR (0.01) accelerated convergence, but in some cases led to oscillations or minor instability.
- Effect of Memory Size (m):
  - Larger memory (15) provided slightly improved Hessian approximations, leading to more stable convergence in some cases.
  - Smaller memory (10) still performed well, but occasionally led to suboptimal updates, particularly when combined with a higher learning rate.

---

3. **Optimal Parameters:**
   - Which combination of learning rate and memory size achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer

- LR = 0.01, Memory Size = 15

This combination achieved the best balance of convergence speed and stability, allowing efficient optimization without excessive oscillations.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying these hyperparameters. For example, did a higher memory size improve convergence at the cost of increased computation time?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer

- Low LR (0.001): Stable but extremely slow, struggling with local minima.
- High LR (0.01): Faster, but required careful tuning to prevent instability.
- Low Memory (10): Worked well but slightly weaker Hessian approximations.
- High Memory (15): Slightly improved updates, but gains were not always significant.

---

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate and memory size when using L-BFGS in similar optimization tasks?

### ✅ Answer

- Use LR = 0.01 for faster convergence but monitor for instability.
- Prefer Memory Size = 15 for better Hessian approximations.
-Avoid excessively low learning rates (0.001) unless stability is a top priority.

---

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the L-BFGS optimizer.


# Nadam (Nesterov-accelerated Adam)

**Mathematical Formulation:**

Nadam combines the ideas of Adam and Nesterov momentum. Its update equations are as follows:

1. **First Moment Update:**

$$
m_t = \beta_1 \, m_{t-1} + (1 - \beta_1) \, \nabla f(w_t)
$$

2. **Second Moment Update:**

$$
v_t = \beta_2 \, v_{t-1} + (1 - \beta_2) \, \left(\nabla f(w_t)\right)^2
$$

3. **Bias Correction:**

$$
\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$

4. **Nesterov Lookahead Term:**

Instead of using \\(\hat{m}_t\\) directly, Nadam uses a combination of the previous moment and the current gradient:

$$
m_{\text{bar}} = \frac{\beta_1 \, m_{t-1} + (1-\beta_1) \, \nabla f(w_t)}{1-\beta_1^t}
$$

5. **Parameter Update:**

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \, m_{\text{bar}}
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate,
- \\(\beta_1\\) and \\(\beta_2\\) are the decay rates for the first and second moments, and
- \\(\epsilon\\) is a small constant for numerical stability.

**Explanation:**

Nadam (Nesterov-accelerated Adam) improves upon Adam by incorporating Nesterov’s accelerated gradient. Instead of updating with just the biased-corrected first moment \\(\hat{m}_t\\), Nadam uses a lookahead estimate \\(m_{\text{bar}}\\) that blends the previous momentum with the current gradient. This “lookahead” helps the optimizer to better anticipate the direction of the parameter update, often leading to faster convergence in practice.


In [58]:
def nadam_update(params, grad, state, hyperparams, **kwargs):
    """
    Nadam update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store moving averages for the first and second moments.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta1": decay rate for the first moment,
            "beta2": decay rate for the second moment,
            "eps": small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Nadam).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr    = hyperparams["lr"]
    beta1 = hyperparams["beta1"]
    beta2 = hyperparams["beta2"]
    eps   = hyperparams["eps"]

    if "m" not in state:
        state["m"] = torch.zeros_like(params)
        state["v"] = torch.zeros_like(params)
        state["t"] = 0

    # Save the previous first moment (for the lookahead term).
    m_prev = state["m"].clone()
    state["t"] += 1

    # Update first moment.
    state["m"] = beta1 * state["m"] + (1 - beta1) * grad
    # Update second moment.
    state["v"] = beta2 * state["v"] + (1 - beta2) * grad**2

    # Bias-corrected moments.
    m_hat = state["m"] / (1 - beta1 ** state["t"])
    v_hat = state["v"] / (1 - beta2 ** state["t"])

    # Compute the lookahead (Nesterov) term:
    m_bar = (beta1 * m_prev + (1 - beta1) * grad) / (1 - beta1 ** state["t"])

    # Update parameters.
    new_params = params - lr * m_bar / (torch.sqrt(v_hat) + eps)
    return new_params, state

In [ ]:
# Define hyperparameters for Nadam.
nadam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Nadam.
run_all_optimizations(update_func=nadam_update, hyperparams=nadam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Nadam.
nadam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Nadam.
run_all_optimizations(update_func=nadam_update, hyperparams=nadam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Nadam.
nadam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Nadam.
run_all_optimizations(update_func=nadam_update, hyperparams=nadam_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for Nadam.
nadam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Nadam.
run_all_optimizations(update_func=nadam_update, hyperparams=nadam_hyperparams, epochs=1000)

# Nadam Hyperparameter Analysis(5 Points)

For the Nadam optimizer, we have defined the initial hyperparameters as follows:

    nadam_hyperparams = {
        "lr": 0.001,      # Learning rate
        "beta1": 0.9,     # Decay rate for first moment
        "beta2": 0.999,   # Decay rate for second moment
        "eps": 1e-8       # Small constant for numerical stability
    }

In this section, you are required to test various combinations of the learning rate, beta1, beta2, and epsilon values to analyze how they affect the performance of Nadam. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the values you tested for the learning rate, beta1, beta2, and epsilon. Provide a clear list or table of these parameter combinations.

### ✅ Answer

| Learning Rate (LR) | Beta1 (β₁) | Beta2 (β₂) |
|--------------------|------------|------------|
| 0.05            | 0.95       | 0.999       |
| 0.05            | 0.9        | 0.990       |
| 0.001            | 0.95       | 0.990       |
| 0.001            | 0.9        | 0.999       |


---

2. **Performance Analysis:**
   - How did different parameter combinations impact the convergence behavior of Nadam?
   - Were there any specific combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested parameter combinations cause the optimizer to get trapped in local minima or exhibit other issues?

### ✅ Answer

- Learning Rate Effects:
  - Higher LR (0.05): Enabled faster convergence but occasionally led to instability.
  - Lower LR (0.001): Ensured stability but significantly slowed down optimization, making convergence inefficient.

- Beta1 and Beta2 Effects:
  - Lower β₁ (0.9) and β₂ (0.990): Allowed for more adaptive updates, improving responsiveness.
  - Higher β₁ (0.95) and β₂ (0.999): Provided smoother updates but sometimes dampened optimization speed, especially with the lower learning rate.

---

3. **Optimal Parameters:**
   - Which combination of parameters achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer

- Best combination: LR = 0.05, β₁ = 0.9, β₂ = 0.990

This configuration offered the best balance of speed and stability, ensuring faster convergence without excessive oscillations or instability. The slightly lower decay rates (β₁ = 0.9, β₂ = 0.990) allowed for a more effective update mechanism.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying these hyperparameters. For example, did lower epsilon or higher beta values improve stability at the cost of slower convergence?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer

- A higher learning rate (0.05) improved convergence but increased the risk of instability with high β values.
- A lower learning rate (0.001) was stable but too slow for practical optimization.
- Higher decay rates (β₂ = 0.999) led to slower adaptation, while slightly lower values (β₂ = 0.990) improved responsiveness.

---

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the learning rate, beta1, beta2, and epsilon when using Nadam in similar optimization tasks?

### ✅ Answer

- Use LR = 0.05 for faster convergence but monitor for instability.
- Set β₁ around 0.9 for a good balance between adaptability and stability.
- Choose β₂ = 0.990 for effective weight updates without excessive damping.
- If facing instability, slightly lower the learning rate or increase β values to smooth updates.

---

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Nadam optimizer.


# AdaDelta

**Mathematical Formulation:**

AdaDelta adaptively adjusts the learning rate by using an exponentially decaying average of past squared gradients and squared updates. The update rules are as follows:

1. **Accumulate Squared Gradients:**

$$
E[g^2]_t = \rho \, E[g^2]_{t-1} + (1-\rho) \, \left(\nabla f(w_t)\right)^2
$$

2. **Compute the Parameter Update:**

$$
\Delta w_t = - \frac{\sqrt{E[\Delta w^2]_{t-1} + \epsilon}}{\sqrt{E[g^2]_t + \epsilon}} \, \nabla f(w_t)
$$

3. **Update the Parameters:**

$$
w_{t+1} = w_t + \Delta w_t
$$

4. **Accumulate Squared Updates:**

$$
E[\Delta w^2]_t = \rho \, E[\Delta w^2]_{t-1} + (1-\rho) \, \left(\Delta w_t\right)^2
$$

where:
- \\( \rho \\) is the decay rate (typically around 0.95 or 0.9),
- \\( \epsilon \\) is a small constant for numerical stability,
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( \nabla f(w_t) \\) is the gradient at \\( w_t \\), and
- \\( E[g^2]_t \\) and \\( E[\Delta w^2]_t \\) are the exponentially decaying averages of squared gradients and updates, respectively.

**Explanation:**

AdaDelta is designed to overcome the aggressive learning rate decay problem of Adagrad by limiting the window of accumulated past gradients using an exponential decay. It does not require an explicit initial learning rate (though you may still set one if desired) and adapts the step size based on the history of gradients and updates. This makes AdaDelta particularly useful for problems where the optimal learning rate changes over time.


In [59]:
def adadelta_update(params, grad, state, hyperparams, **kwargs):
    """
    AdaDelta update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state variables.
          Expected keys:
            "Eg2": Exponential moving average of squared gradients.
            "Edw2": Exponential moving average of squared updates.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "rho": Decay rate (e.g., 0.95),
            "eps": A small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for AdaDelta).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    rho = hyperparams["rho"]
    eps = hyperparams["eps"]

    # Initialize state if not present.
    if "Eg2" not in state:
        state["Eg2"] = torch.zeros_like(params)
    if "Edw2" not in state:
        state["Edw2"] = torch.zeros_like(params)

    # Update the exponential moving average of squared gradients.
    state["Eg2"] = rho * state["Eg2"] + (1 - rho) * grad**2

    # Compute the update:
    # Note: Classic AdaDelta uses an effective learning rate of 1.
    update = - (torch.sqrt(state["Edw2"] + eps) / torch.sqrt(state["Eg2"] + eps)) * grad

    # Update parameters.
    new_params = params + update

    # Update the exponential moving average of squared updates.
    state["Edw2"] = rho * state["Edw2"] + (1 - rho) * update**2

    return new_params, state

In [ ]:
# Define hyperparameters for AdaDelta.
adadelta_hyperparams = {
    "rho": 0.95,    # Decay rate
    "eps": 1e-6     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using AdaDelta.
run_all_optimizations(update_func=adadelta_update, hyperparams=adadelta_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for AdaDelta.
adadelta_hyperparams = {
    "rho": 0.90,    # Decay rate
    "eps": 1e-6     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using AdaDelta.
run_all_optimizations(update_func=adadelta_update, hyperparams=adadelta_hyperparams, epochs=1000)

In [ ]:
# Define hyperparameters for AdaDelta.
adadelta_hyperparams = {
    "rho": 0.999,    # Decay rate
    "eps": 1e-6     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using AdaDelta.
run_all_optimizations(update_func=adadelta_update, hyperparams=adadelta_hyperparams, epochs=1000)

# Adadelta Hyperparameter Analysis

For the Adadelta optimizer, we have defined the initial hyperparameters as follows:

    adadelta_hyperparams = {
        "rho": 0.95,    # Decay rate
        "eps": 1e-6     # Small constant for numerical stability
    }

In this section, you are required to test various values for the decay rate (rho) and epsilon to analyze how they affect the performance of Adadelta. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - List all the values you tested for rho and epsilon. Provide a clear list or table of these parameter combinations.

### ✅ Answer

| Decay Rate  |
|-------------|
| 0.90        |
| 0.95        |
| 0.990        |

---

2. **Performance Analysis:**
   - How did different values of rho and epsilon impact the convergence behavior of Adadelta?
   - Were there any specific parameter combinations that resulted in unstable behavior, overshooting, or slow convergence?
   - Did any of the tested combinations cause the optimizer to get stuck in local minima or exhibit other issues?

### ✅ Answer

**Effect of Decay Rate**: A lower decay rate of 0.90 led to more responsive updates, which helped in dynamic optimization landscapes but introduced some instability. Increasing ρ to 0.95 provided a more balanced approach, maintaining stability while allowing sufficient adaptation. The highest decay rate, 0.999, resulted in smoother but slower updates, leading to more stable convergence at the cost of adaptability in complex functions.

---

3. **Optimal Parameters:**
   - Which combination of rho and epsilon achieved the best balance between convergence speed and stability?
   - Explain why you consider these parameters optimal based on your observations.

### ✅ Answer

The decay rate of 0.95 emerged as the best choice, striking a balance between learning efficiency and stability. It prevented excessive noise while ensuring sufficient adaptation, making it a more reliable default option.

---

4. **Trade-offs and Observations:**
   - Discuss any trade-offs you noticed when varying rho and epsilon. For example, did a higher rho improve convergence stability at the cost of slower updates?
   - Describe any patterns or trends that emerged from your experiments.

### ✅ Answer

Lower decay rates improved responsiveness but made convergence noisier. Higher values helped maintain stability but slowed down optimization. Since AdaDelta does not require manual learning rate tuning, the primary tuning parameter was ρ, whose impact was relatively minor within the tested range. The optimizer performed well on simpler functions but struggled in more complex, multimodal landscapes.

----

5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the decay rate and epsilon when using Adadelta in similar optimization tasks?

### ✅ Answer

A decay rate of 0.95 is generally preferable for AdaDelta as it provides a good balance of stability and adaptability. If the loss function fluctuates significantly, a lower value like 0.90 may help in faster adaptation. Conversely, for highly stable optimization scenarios, 0.999 may be considered to ensure smoother updates. However, for more challenging optimization problems, alternative adaptive optimizers like Adam or RMSprop may offer better results.

---

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adadelta optimizer.


# Final Section

I hope you enjoyed implementing these concepts. For the final submission, you are required to integrate your explanations regarding the algorithms discussed above. You simply need to elaborate on the points provided here.

Best of luck!
